<img src="http://www.cidaen.es/assets/img/mCIDaeNnb.png" alt="Logo CiDAEN" align="right">

<h1><font size=4>Trabajo Fin de Master (TFM)</font></h1>
<br>
<h2><font size=6>WiDS Datathon 2024 - Challenge 2</font></h2>
<h3><font size=5>Modelos de regresión para estimación del periodo de diagnóstico metastático</font></h3>
<h3><font size=5>Parte 2 - Modelos de Regresión</font></h3>
<br>
<h1><font size=4>Alumna: Luna Jiménez Fernández</font></h1>
<br>



<div align="right">
<font size=3>Máster en Ciencia de Datos e Ingeniería de Datos en la Nube</font><br>
<font size=3>Universidad de Castilla-La Mancha</font>
</div>

<br>

---

In [109]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

# Python imports
from time import time
import itertools

# Array manipulation libraries
import numpy as np
import pandas as pd

# Pre-processing
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Simple regression models
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import LinearSVR, SVR

# Bagging ensemble models
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor

# Boosting ensemble models
from sklearn.ensemble import AdaBoostRegressor, HistGradientBoostingRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

# Experimentation
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical

# Model saving and loading
import pickle
import json
import os
import os.path
from pathlib import Path

# Importing visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [ ]:
# Seed for random experiments - 7 is the number
RANDOM_SEED = 777

En la primera libreta se realizó un **análisis exploratorio de datos** exhaustivo para entender en profundidad el comportamiento del conjunto de datos de interés - el **segundo desafío** del **Women in Data Science (*WiDS*) Datathon** del año 2024, disponible en el [siguiente enlace](https://www.kaggle.com/competitions/widsdatathon2024-challenge2/overview).

Tras este estudio, el objetivo de la siguiente libreta es tanto la **construcción de modelos de regresión** capaces de predecir el **tiempo de diagnóstico de la metástasis** a partir de los atributos seleccionados, como la **evaluación** de estos con el fin de estudiar si resultan de utilidad y si - como se planteó - los **atributos geográficos, socioeconómicos y climáticos** juegan algún papel relevante en la estimación de los valores.

---

# Índice

* [3. Selección de atributos y pre-procesamiento](#section3)
    * [3.1. Carga y particionamiento del conjunto de datos](#section3-1)
    * [3.2. Selección de atributos](#section3-2)
    * [3.3. Pre-procesamiento de los datos](#section3-3)
* [4. Selección de modelos de regresión e hiperparámetros](#section4)
    * [4.1. Baselines - estimadores lineales, árboles y SVMs](#section4-1)
    * [4.2. Ensembles - estimadores basados en agrupaciones de estimadores simples](#section4-2)
* [5. Experimentación](#section5)
* [6. Análisis de resultados](#section6)
* [7. Conclusiones](#section7)
---

<a id="section3"></a>

# 3. Selección de atributos y pre-procesamiento

Tras finalizar el **análisis exploratorio de datos** en la libreta anterior, el siguiente paso en el proceso de ciencia de datos es el **preprocesamiento de la información** - para ser utilizada posteriormente por modelos de regresión, con el fin de predecir el tiempo de diagnóstico de la metástasis.

Concretamente, en este apartado se realizan las siguientes preparaciones:
- **Cargar y particionar** los conjuntos de datos en **entrenamiento**, **validación** y **test**.
- **Seleccionar el subconjunto de atributos** que van a ser utilizados durante la experimentación.
- **Preparar las *pipelines*** encargadas de transformar los datos crudos en datos listos para ser utilizados por los modelos posteriores.

---

<a id="section3-1"></a>

## 3.1. Carga y particionamiento del conjunto de datos

Durante el análisis exploratorio de datos se trabajó únicamente sobre el **conjunto de entrenamiento** - con el fin de evitar cualquier posible fuga de datos al estudiar el conjunto de test. Ahora bien, el desafio en Kaggle ofrece **dos conjuntos de datos**:
- `train.csv`: El **conjunto de entrenamiento**, con **150 atributos** y los valores de la **variable objetivo** (el tiempo de diagnóstico) asociados a cada instancia.
- `test.csv`: El **conjunto de test**, conteniendo únicamente los **150 atributos** sin los valores de la variable objetivo.

El primer paso, por tanto, consiste en **cargar ambos conjuntos de datos** como *DataFrames*:

In [3]:
# Loading the CSV files - force zip3 to be read as a string
df_train = pd.read_csv("data/train.csv", index_col="patient_id", dtype={"patient_zip3": object})
df_test = pd.read_csv("data/test.csv", index_col="patient_id", dtype={"patient_zip3": object})

# Display a small sample of both datasets to show that they have been properly loaded
print(f"Training size: {df_train.shape}")
display(df_train.sample(5))
print(f"Test size: {df_test.shape}")
display(df_test.sample(5))

Training size: (13173, 151)


,patient_race,payer_type,patient_state,patient_zip3,Region,Division,patient_age,patient_gender,bmi,breast_cancer_diagnosis_code,...,Average of Apr-18,Average of May-18,Average of Jun-18,Average of Jul-18,Average of Aug-18,Average of Sep-18,Average of Oct-18,Average of Nov-18,Average of Dec-18,metastatic_diagnosis_period
patient_id,,,,,,,,,,,,,,,,,,,,,
140631,NaN,COMMERCIAL,TN,373,South,East South Central,72,F,NaN,C50411,...,53.50,71.31,75.20,76.96,75.78,74.87,61.06,44.31,42.83,124
837772,NaN,COMMERCIAL,CA,957,West,Pacific,60,F,NaN,C50919,...,54.84,62.45,69.98,79.61,74.98,71.08,63.07,52.90,44.10,0
892031,NaN,COMMERCIAL,IL,605,Midwest,East North Central,64,F,NaN,C50812,...,40.21,66.45,71.55,74.22,74.33,67.92,51.31,33.70,31.66,0
204277,NaN,NaN,CA,939,West,Pacific,64,F,NaN,C50411,...,57.74,60.36,64.18,69.50,66.94,63.46,63.23,57.98,51.67,10
971648,NaN,COMMERCIAL,CA,956,West,Pacific,83,F,34.18,C50919,...,56.86,63.99,70.91,78.12,73.81,70.75,64.60,54.03,45.47,33


Test size: (5646, 150)


,patient_race,payer_type,patient_state,patient_zip3,Region,Division,patient_age,patient_gender,bmi,breast_cancer_diagnosis_code,...,Average of Mar-18,Average of Apr-18,Average of May-18,Average of Jun-18,Average of Jul-18,Average of Aug-18,Average of Sep-18,Average of Oct-18,Average of Nov-18,Average of Dec-18
patient_id,,,,,,,,,,,,,,,,,,,,,
374317,White,NaN,OH,450,Midwest,East North Central,47,F,NaN,1749,...,37.76,46.06,71.84,74.79,76.20,76.77,71.75,56.15,38.76,35.72
940429,NaN,COMMERCIAL,NY,111,Northeast,Middle Atlantic,40,F,NaN,C50511,...,37.95,47.68,65.17,70.33,77.22,77.83,69.93,55.92,42.16,37.26
857682,White,NaN,KY,404,South,East South Central,38,F,NaN,1743,...,43.81,51.62,71.46,73.65,75.23,75.27,73.01,59.63,43.57,41.00
590257,Black,NaN,IA,527,Midwest,West North Central,30,F,NaN,C50912,...,35.71,41.20,68.76,73.85,74.00,73.74,67.13,49.89,31.92,31.06
277660,White,MEDICAID,NY,128,Northeast,Middle Atlantic,59,F,NaN,1749,...,30.02,36.93,58.30,61.44,70.73,69.72,62.24,46.36,32.19,26.42


Ahora bien, debido al proceso que se va a seguir durante el entrenamiento de los modelos (**selección de hiperparámetros**, **selección de modelos** y **evaluación**), utilizar directamente los conjuntos de datos descritos podría llevar a un problema de **fuga de datos** - al usar el mismo conjunto de datos para entrenar los modelos y evaluar sus hiperparámetros.

Para evitar esto, se va a dividir el conjunto de entrenamiento en dos - un conjunto de **entrenamiento** y uno de **validación**  -, siendo la distribución final la siguiente:
- **Entrenamiento**: El conjunto de entrenamiento cumple dos tareas - tanto el **entrenamiento de los modelos de regresión** propuestos como el **ajuste de hiperparámetros de los mismos** a través de una validación cruzada.
- **Validación**: Una vez se tienen los modelos entrenados, el conjunto de validación será utilizado para **seleccionar el mejor modelo de forma honesta** - utilizando un conjunto de datos que no han utilizado durante el entrenamiento para evitar sesgos o fugas de datos.
- **Test**: Finalmente, se utilizará el conjunto de test para **evaluar el rendimiento real** del modelo seleccionado a través de la validación - utilizando una plataforma externa (***Kaggle***) para medir este rendimiento. 

Además, todos estos conjuntos de datos se van a fraccionar en **atributos** (`X`) y **variable objetivo** (`y`) para seguir el estándar de `scikit-learn`.

In [4]:
# Split the datasets into training, validation and test
# Train / Val
X, y = df_train.drop(columns="metastatic_diagnosis_period"), df_train["metastatic_diagnosis_period"]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=RANDOM_SEED)

# Test
X_test = df_test

# Display the information about each dataset - to ensure that it has been loaded and partitioned correctly
print("ENTRENAMIENTO:")
print(f"\t-Atributos: {X_train.shape}")
print(f"\t-Variable objetivo: {y_train.shape}")
print("VALIDACIÓN:")
print(f"\t-Atributos: {X_val.shape}")
print(f"\t-Variable objetivo: {y_val.shape}")
print("TEST:")
print(f"\t-Atributos: {X_test.shape}")

ENTRENAMIENTO:
	-Atributos: (9879, 150)
	-Variable objetivo: (9879,)
VALIDACIÓN:
	-Atributos: (3294, 150)
	-Variable objetivo: (3294,)
TEST:
	-Atributos: (5646, 150)


---

<a id="section3-2"></a>

## 3.2. Selección de atributos

Como se observó durante el análisis exploratorio, no tendría sentido utilizar directamente el **conjunto de datos completos** para el entrenamiento de modelos:
- La **dimensionalidad del conjunto de datos** - con 150 atributos en total - es excesiva para la cantidad de datos disponible, lo que podría llevar a sobreajustes.
- Algunos atributos tienen **una cantidad excesiva de posibles valores** - que se puede traducir, de nuevo, en sobreajustes del modelo al no tener suficientes datos para aprender adecuadamente las relaciones.
- La **amplia mayoría de atributos son irrelevantes** para la variable objetivo - ya sea por su baja calidad o por la poca correlación que tienen con la variable objetivo.

Por tanto, es necesario **seleccionar un subconjunto de atributos**, con el fin reducir la dimensionalidad y cribar los atributos que no sean relevantes para la predicción. Para buscar este subconjunto, se proponen varias posibilidades - almacenadas en el diccionario `attribute_selection`:

In [5]:
attribute_selection = {}

### 3.2.1. Selección por análisis exploratorio

Durante el análisis exploratorio de datos se ha realizado un análisis exhaustivo de los datos - tanto su **comportamiento** como su **relevancia** y las **transformaciones** que serían necesarias para utilizarse.

A partir de las conclusiones extraidas, se ha obtenido el siguiente **conjunto de atributos** - representando los atributos más relevantes estudiados dentro del conjunto de datos, junto a las **transformaciones a aplicar** sobre éstos:
- **Código de diagnóstico del cancer de mama (`breast_cancer_diagnosis_code`):** Variable categórica.
    - Debido al gran número de posibles valores, es necesario **agrupar los valores menos frecuentes**.
- **Código de diagnóstico del cancer metastático (`metastatic_cancer_diagnosis_code`):** Variable categórica.
    - Debido al gran número de posibles valores, es necesario **agrupar los valores menos frecuentes**.
- **Estado de residencia del paciente (`patient_state`):** Variable categórica.
    - Debido al gran número de posibles valores, es necesario **agrupar los valores menos frecuentes**.
- **Raza del paciente (`patient_race`):** Variable categórica.
    - Se **agrupan los valores perdidos** bajo un único valor.
- **Tipo de seguro médico del paciente (`payer_type`):** Variable categórica.
    - Se **agrupan los valores perdidos** bajo un único valor.

Como se puede observar, **todos los atributos seleccionados son categóricos**.

In [6]:
attribute_selection["manual"] = {
    "categorical": ["breast_cancer_diagnosis_code", "metastatic_cancer_diagnosis_code", "patient_state", "patient_race", "payer_type"]
}

Estas transformaciones se han elegido en base a los **test estadísticos** que se realizaron durante el análisis exploratorio:
- La **agrupación de los valores** en las variables de alta dimensionalidad aumenta la significación estadística, al reducirse el número de valores con un número demasiado bajo de instancias.
- La **sustitución de valores perdidos** mejora el rendimiento en los atributos donde el número de valores perdidos es excesivo.

A su vez, se ha optado por descartar los siguientes atributos:
- **Edad (`patient_age`) y IMC (`bmi`) del paciente**: Variables numéricas sin correlación con la variable objetivo.
- **Región (`Region`) y división (`Division`) del paciente**: Variables categóricas con poca relevancia, y ya representadas por otra variable más significativa (`patient_state`).
- **Código zip del paciente (`patient_zip3`)**: Variable categórica ya representada por otra variable (`patient_state`) con dimensionalidad excesiva.
- **Todas las variables geográficas, socioeconómicas y climáticas**: 136 atributos numéricos sin correlación con la variable objetivo.

### 3.2.2. Selección automática

En el apartado anterior se ha obtenido un subconjunto de atributos en base al estudio que se realizó en la libreta anterior. Ahora bien, aunque el estudio estuviese apoyado en **gráficas y tests estadísticos**, sigue existiendo la posibilidad de que **el analista haya introducido sesgos propios**.

Otra posibilidad es utilizar **algoritmos de aprendizaje automático** para realizar el proceso de selección de atributos - ya sea basandose en **tests estadísticos** o en el **comportamiento de modelos reales entrenados sobre los datos**.

Ahora bien, la mayoría de métodos necesitan un **preprocesamiento previo**, para imputar valores perdidos y unificar el comportamiento de atributos categóricos y numéricos. Concretamente, en este caso:
- **Atributos categóricos**: Se imputan los valores perdidos como un valor constante (`UNKNOWN`) y se **codifican los posibles valores del atributo** utilizando `One-Hot Encoding` - donde cada valor de cada variable categórica se **representa como un atributo separado**, que puede ser `True` (si el valor de la variable se corresponde) o `False` en cualquier otro caso.
    - Debido a la gran complejidad de algunos atributos, se opta por **agrupar todos los valores con menos de 100 instancias**. Este valor se ha elegido de forma arbitraria para la selección de variables, y tendrá que ser ajustado posteriormente durante el ajuste de hiperparámetros.
    - La codificación va a causar que **aumente el número de atributos respecto al conjunto de entrenamiento original** - lo que significa que **será necesario re-agrupar los valores de las variables mediante estadísticos** para realizar la selección.
- **Atributos numéricos**: Se imputan los valores perdidos utilizando la **mediana** (debido al gran número de valores extremos) y se **escalan los valores** utilizando la mediana y la desviación estándar.

Se define un `Pipeline` para realizar automáticamente el preprocesamiento descrito - aplicandose sobre los datos del **conjunto de entrenamiento** para evitar fugas de datos:

In [7]:
# Extract all attributes of each type
categorical_variables = X_train.select_dtypes(exclude=np.number).columns.to_list()
numerical_variables = X_train.select_dtypes(include=np.number).columns.to_list()

# Feature Selection pipeline - Transforms the dataset to allow for feature selection
fs_preprocessing = ColumnTransformer([
    # Categorical attributes - imputting with a constant value and encoding (one-hot)
    ("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="constant", fill_value="UNKNOWN")),
        ("oh", OneHotEncoder(min_frequency=100))
    ]), categorical_variables),
    # Numerical attributes - imputting with median values and scaling
    ("num", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", RobustScaler())
    ]), numerical_variables)
])

# Display the pre-processing pipeline
display(fs_preprocessing)

ColumnTransformer(transformers=[('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='UNKNOWN',
                                                                strategy='constant')),
                                                 ('oh',
                                                  OneHotEncoder(min_frequency=100))]),
                                 ['patient_race', 'payer_type', 'patient_state',
                                  'patient_zip3', 'Region', 'Division',
                                  'patient_gender',
                                  'breast_cancer_diagnosis_code',
                                  'breast_cancer_diagnosis_desc',
                                  'metastatic_cancer_diagnos...
                                  'age_60s', 'age_70s', 'age_over_80', 'male',
                                  'female', 'married', 'divorced',
                                  'never_married', 'widowed', 'family_size',
                                  'family_dual_income',
                                  'income_household_median',
                                  'income_household_under_5',
                                  'income_household_5_to_10',
                                  'income_household_10_to_15',
                                  'income_household_15_to_20',
                                  'income_household_20_to_25',
                                  'income_household_25_to_35',
                                  'income_household_35_to_50', ...])])

Existen dos estrategias para realizar una selección automática de atributos:

#### - Métodos *filter*

Los **métodos de filtrado (*filter*)** utilizan **test estadísticos** para evaluar de forma automática la relevancia de cada variable, sin la necesidad de entrenar modelos- haciendolos **más agiles**, aunque menos capaces de identificar las correlaciones entre grupos de atributos.

Para este caso se utilizan **tests F** - tests de **varianza**, para identificar las **diez variables** más relevantes del conjunto de datos 

In [8]:
# FEATURE RANKING - Using F-Regression
# Create the pipeline
fs_kbest = Pipeline([
    ("preprocessing", fs_preprocessing),
    ("kbest", SelectKBest(f_regression, k="all"))
])
fs_kbest.fit(X_train, y_train)

# Join the variable names and values
df_kbest = pd.DataFrame({
    "variable": fs_kbest.get_feature_names_out(),
    "score": fs_kbest["kbest"].scores_
})

# Process the dataframe, step by step
df_kbest_ordered = (
    df_kbest.assign(variable=(
        df_kbest["variable"].str.extract(r"cat__(?P<cat>[0-9A-Za-z_\- ]+)_(?:[0-9A-Za-z\- ]+|infrequent)|num__(?P<num>[0-9A-Za-z_\- ]+)")   # 1 - Extract the proper variable name
        .apply(lambda s: s["cat"] if not pd.isna(s["cat"]) else s["num"], axis=1)                                                           #     (joining into a single column)
    ))
    .groupby("variable").agg(max_score=("score", "max"), avg_score=("score", "mean"))                                                       # 2 - Obtain the maximum and avg score of each variable
    .sort_values(by="max_score", ascending=False)                                                                                           # 3 - Sort by MAXIMUM score
    
)

# Display the 15 best attributes
display(df_kbest_ordered.head(15))

,max_score,avg_score
variable,,
breast_cancer_diagnosis_desc,3163.104702,293.539683
breast_cancer_diagnosis_code,3163.104702,293.539683
metastatic_cancer_diagnosis_code,58.998175,12.662202
payer_type,56.152011,21.747065
patient_age,32.205066,32.205066
breast_cancer_diagnosis_desc_infrequent,30.799057,30.799057
breast_cancer_diagnosis_code_infrequent,30.799057,30.799057
patient_state,28.378242,3.208365
labor_force_participation,15.166246,15.166246


A partir de estos atributos obtenidos, se observa que:
- Los **atributos más importantes** son consistentes con los observados durante el estudio: **el código de diagnóstico de cancer** - tanto original como metástasis - y el **tipo de seguro médico**.
- El algoritmo da **mayor importancia** a algunos atributos numéricos - principalmente:
    - **Edad del paciente** (`patient_age`)
    - **Porcentaje de residentes empleados** (`labor_force_participation`)
    - **Porcentaje de residentes con estudios universitarios** (`education_bachelors` y `education_college_or_above`)
    - **Porcentaje de hogares con dos o más ingresos** (`family_dual_income`)
- Algunos atributos categóricos estudiados **tienen menor importancia de la esperada**:
    - **Estado de residencia del paciente** (`patient_state`)
    - **Raza del paciente** (`patient_race`)

Es de interés destacar que **existe una diferencia muy considerable en la importancia de los atributos** - donde el **tipo de cancer de mama original** es varios ordenes de magnitud más relevante que el resto de variable.

El **subconjunto de variables** obtenido por el proceso es el siguiente:

In [9]:
attribute_selection["filter"] = {
    "categorical": ["breast_cancer_diagnosis_code", "metastatic_cancer_diagnosis_code", "payer_type", "patient_state", "patient_race"],
    "numerical": ["patient_age", "labor_force_participation", "education_bachelors", "family_dual_income", "education_college_or_above"]
}

#### - Métodos *wrapper*

Los **métodos de envoltura *wrapper*** utilizan **modelos de aprendizaje automáticos** entrenados sobre los datos para elegir las variables más relevantes. Pese a ser más lentos, sus resultados son **más fiables** - al estar estudiando el comportamiento real de un modelo.

Para este caso se entrenará un modelo de **Random Forest**, utilizando el **conjunto de entrenamiento** para evitar fugas - buscando encontrar los **diez atributos más relevantes**:

In [10]:
# FEATURE RANKING - Using Random Forest as a wrapper
# Create the pipeline
fs_rf = Pipeline([
    ("preprocessing", fs_preprocessing),
    ("wrapper", RandomForestRegressor(n_estimators=100, random_state=RANDOM_SEED))
])
fs_rf.fit(X_train, y_train)

# Join the variable names and values
df_rf = pd.DataFrame({
    "variable": fs_rf["preprocessing"].get_feature_names_out(),
    "score": fs_rf["wrapper"].feature_importances_
})

# Process the dataframe, step by step
df_rf_ordered = (
    df_rf.assign(variable=(
        df_rf["variable"].str.extract(r"cat__(?P<cat>[0-9A-Za-z_\- ]+)_(?:[0-9A-Za-z\- ]+|infrequent)|num__(?P<num>[0-9A-Za-z_\- ]+)")      # 1 - Extract the proper variable name
        .apply(lambda s: s["cat"] if not pd.isna(s["cat"]) else s["num"], axis=1)                                                           #     (joining into a single column)
    ))
    .groupby("variable").agg(max_score=("score", "max"), avg_score=("score", "mean"))                                                       # 2 - Obtain the maximum and avg score of each variable
    .sort_values(by="max_score", ascending=False)                                                                                           # 3 - Sort by MAXIMUM score
    
)

# Display the 15 best attributes
display(df_rf_ordered.head(15))

,max_score,avg_score
variable,,
breast_cancer_diagnosis_code,0.123410,0.012368
breast_cancer_diagnosis_desc,0.119221,0.012102
patient_age,0.079427,0.079427
bmi,0.037954,0.037954
metastatic_cancer_diagnosis_code,0.012096,0.004549
breast_cancer_diagnosis_desc_infrequent,0.010563,0.010563
breast_cancer_diagnosis_code_infrequent,0.009509,0.009509
payer_type,0.007699,0.006449
patient_race,0.007403,0.005098


*(NOTA: `RandomForest` tiene un componente aleatorio. Se ha utilizado una semilla, pero sigue existiendo la posibilidad de que cambien los valores en ejecuciones posteriores)*

A partir de estos atributos, se observa que:

- En este caso, el atributo más importante es **únicamente el código de diagnóstico del cancer de mama** - el diagnóstico de metástasis **pierde peso**.
- Tienen un mayor peso los **atributos numéricos**:
    - **Edad** (`patient_age`) y **IMC** (`bmi`) del paciente.
    - **Mediana del tiempo de transporte de los residentes al trabajo** (`commute_time`).
    - **Porcentaje de la población con edad superior a 40 años** (`age_40s`).
    - **Porcentaje de la población con estudios en STEM** (`education_stem_degree`).
    - **Porcentaje de la población identificada con razas nativas** (`race_native`).
- Los **atributos categóricos identificados** - **raza del paciente** (`patient_race`) y **tipo de seguro médico** (`payer_type`) pierden importancia.
- **El estado de residencia del paciente** (`patient_state`) deja de estar entre los **diez atributos más relevantes**.

El subconjunto de atributos obtenidos por este proceso es el siguiente:

In [11]:
attribute_selection["wrapper"] = {
    "categorical": ["breast_cancer_diagnosis_code", "metastatic_cancer_diagnosis_code", "patient_race", "payer_type"],
    "numerical": ["patient_age", "bmi", "commute_time", "race_native", "education_stem_degree", "age_40s"]
}

### 3.2.3. Conjunto de datos completo

Finalmente, otra opción posible es **utilizar todos los atributos del conjunto de datos** sin ningún tipo de criba o pre-selección. Esta opción tiene ciertas **ventajas** y **desventajas**:
- Por un lado, **algunos modelos funcionan mejor sin selección de variables** - especialmente aquellos que realizan una selección interna (como la **regresión lineal de tipo Lasso** o los **ensembles tipo Random Forest**).
- Por otro lado, **la complejidad excesiva del conjunto de datos** puede afectar al entrenamiento - provocando sobreajustes por la falta de instancias en el conjunto de datos, y perjudicando al rendimiento de los modelos sin selección de atributos interna. Además, **el tiempo de entrenamiento aumenta considerablemente** debido a la mayor complejidad de los modelos entrenados.

En este caso, se incluye principalmente como ***baseline*** para el resto de opciones - pudiendo de esta forma estudiar si **la selección de variables ha mejorado el rendimiento de los modelos**.

In [12]:
attribute_selection["no_selection"] = {
    "categorical": X_train.select_dtypes(exclude=np.number).columns.to_list(),
    "numerical": X_train.select_dtypes(include=np.number).columns.to_list()
}

---

<a id="section3-3"></a>

## 3.3. Pre-procesamiento de los datos

El último paso antes del entrenamiento de los modelos es el **preprocesamiento del conjunto de datos** - realizar las transformaciones adecuadas sobre el conjunto de datos para **optimizarlo** de cara al entrenamiento y evaluación de los modelos, con el fin de mejorar su rendimiento.

Los pasos que se van a seguir para pre-procesar los datos - dependiendo del **tipo de datos** - son los siguientes:
- **Atributos categóricos**:
    1. **Imputación**: Se reemplazan todos los valores perdidos por un **valor constante** - `UNKNOWN`.
        - Como se estudió durante el análisis exploratorio, en la mayoría de variables categóricas **tiene sentido tratar los valores perdidos como un valor distinto** - al poder representar valores que no se han podido adquirir o que no se han querido compartir.
    2. **Codificación**: Se transforma la representación de la variable utilizando **`One-Hot Encoding`** - el atributo se divide en **tantos atributos como valores tiene la variable**, donde cada uno de estos nuevos atributos representa si la instancia contiene el valor representado (`1`) o no (`0`).
        - Debido a la gran complejidad de los atributos categóricos (teniendo, en general, **40 o más posibles valores**) y a que **los atributos no son exhaustivos** y pueden haber valores no vistos antes en el conjunto de datos, es necesario **agrupar los valores menos frecuentes** para reducir la dimensionalidad. El **umbral de agrupamiento** será uno de los hiperparámetros a ajustar posteriormente.
        - Es importante destacar que **los métodos de *Gradient Boosting* no necesitan codificación de las variables categóricas** - por lo que este paso es opcional en dichos casos.
- **Atributos numéricos**:
    1. **Imputación**: Se reemplazan todos los valores perdidos por **la mediana del atributo**.
        - En general, el conjunto de datos tiene un gran número de **valores extremos y sesgos**. Utilizar la mediana frente a la media ayuda a crear valores más robustos.
    2. **Escalado**: Se *centran* los valores de los atributos alrededor de la **mediana** y la **desviación estándar**.
        - Normalmente se escala utilizando el **valor promedio**. Ahora bien, para tener un escalado más robusto se utiliza el **rango intercuartil** para el escalado a través de la clase `RobustScaler`.

El problema ahora es que los pasos de preprocesamiento deben ser **idénticos y replicables** en todos los aspectos - tanto entre los experimentos con **distintos modelos y subconjuntos de atributos**, como entre **el entrenamiento y la evaluación** con distintos conjuntos de datos.

Para automatizar este proceso, se definen **`Pipelines`** - cadenas de transformaciones que se aplican **automáticamente** antes de usar los modelos de aprendizaje automático, tanto para **entrenarlos** como para **predecir la variable objetivo para un conjunto de datos**. 

Debido al funcionamiento de `scikit-learn`, es necesario definir `Pipelines` para cada uno de los subconjuntos de atributos que se van a utilizar. Estos pasos de preprocesamiento se almacenan en el diccionario `preprocessing_pipelines` para su posterior automarización, con la siguiente estructura:

```
preprocessing_pipelines{
    "one-hot": {
        <nombre de subconjunto de atributos>: Pipeline
        ...
    },
    "unmodified": {
        <nombre de subconjunto de atributos>: Pipeline
        ...
    }
}
```

Debido a la diferencia en el funcionamiento de los modelos a utilizar durante la experimentación - concretamente, al **tratamiento de los atributos categóricos** -, se pueden distinguir dos familias de *Pipelines*:
- `one-hot`: *Pipelines* que aplican un **proceso de codificación *one-hot*** sobre los atributos categóricos - la mayoría de modelos.
- `unmodified`: *Pipelines* que **no aplican ninguna transformación a los atributos categóricos** más allá de la imputación de valores perdidos - concretamente los modelos de *Gradient Boosting*.

In [13]:
# Prepare the dictionary 
preprocessing_pipelines = {
    "one-hot": {},
    "unmodified": {}
}

#### - Subconjunto manual (sin variables numéricas)

La selección manual de atributos solo ha incluido atributos categóricos. Por tanto, sus `Pipelines` correspondientes **no necesitan incorporar transformaciones relativas a atributos numéricos**.

Ahora bien, para facilitar la automatización posterior de los experimentos, se **utilizan `ColumnTransformer`** pese a no haber separación por tipos de datos - al seguir la misma estructura que el resto de *Pipelines*, no es necesario tratarlo de forma distinta.

In [14]:
# MANUAL PIPELINE - Categorical values
# Reuses the ColumnTransformer structure to automatize accessing the OneHotEncoder hyperparameters - as min_frequency will be adjusted
# Otherwise, the code would have to take into account the subset of attributes being used to know the inner pipeline structure

# With one-hot encoding
preprocessing_pipelines["one-hot"]["manual"] = ColumnTransformer([
    # Imput and encode categorical attributes
    ("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="constant", fill_value="UNKNOWN")),
        ("oh", OneHotEncoder(handle_unknown="infrequent_if_exist", min_frequency=100))
    ]), attribute_selection["manual"]["categorical"])
])

# Without encoding
preprocessing_pipelines["unmodified"]["manual"] = ColumnTransformer([
    # Imput categorical attributes
    ("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="constant", fill_value="UNKNOWN"))
    ]), attribute_selection["manual"]["categorical"])
])

### - Subconjuntos automáticos (*filter* y *wrapper*) y conjunto de datos completo

En todos estos casos, la selección de atributos ha incluido atributos **categóricos y numéricos**. Por tanto, el *Pipeline* debe ser capaz de procesar ambos tipos de atributos por separado - para lo que se utiliza `ColumnTransformer`:

In [ ]:
# CATEGORICAL AND NUMERICAL PIPELINES
# Feature selection subsets to consider
fs_list = ["filter", "wrapper", "no_selection"]

for feature_subset in fs_list:

    # Preprocessing with one-hot categorical attribute encoding
    preprocessing_pipelines["one-hot"][feature_subset] = ColumnTransformer([
        # Imput and encode categorical attributes
        ("cat", Pipeline([
            ("imputer", SimpleImputer(strategy="constant", fill_value="UNKNOWN")),
            ("oh", OneHotEncoder(handle_unknown="infrequent_if_exist", min_frequency=100))
        ]), attribute_selection[feature_subset]["categorical"]),
        # Imput and scale numerical attributes
        ("num", Pipeline([
            ("imputer", SimpleImputer(strategy="median")),
            ("scaler", RobustScaler())
        ]), attribute_selection[feature_subset]["numerical"])
    ])

    # Preprocessing without categorical attribute encoding
    preprocessing_pipelines["unmodified"][feature_subset] = ColumnTransformer([
        # Imput categorical attributes
        ("cat", Pipeline([
            ("imputer", SimpleImputer(strategy="constant", fill_value="UNKNOWN")),
        ]), attribute_selection[feature_subset]["categorical"]),
        # Imput and scale numerical attributes
        ("num", Pipeline([
            ("imputer", SimpleImputer(strategy="median")),
            ("scaler", RobustScaler())
        ]), attribute_selection[feature_subset]["numerical"])
    ])

Para comprobar que se ha realizado la construcción de *Pipelines* de forma adecuada, se muestran **los pipelines resultantes**:

In [ ]:
# Display every preprocessing pipeline
for encoding_type, encoding_dict in preprocessing_pipelines.items():
    for feature_subset, pipeline in encoding_dict.items():
        print(f"{encoding_type} - {feature_subset}")
        display(pipeline)

one-hot - manual


ColumnTransformer(transformers=[('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='UNKNOWN',
                                                                strategy='constant')),
                                                 ('oh',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist',
                                                                min_frequency=100))]),
                                 ['breast_cancer_diagnosis_code',
                                  'metastatic_cancer_diagnosis_code',
                                  'patient_state', 'patient_race',
                                  'payer_type'])])

one-hot - filter


ColumnTransformer(transformers=[('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='UNKNOWN',
                                                                strategy='constant')),
                                                 ('oh',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist',
                                                                min_frequency=100))]),
                                 ['breast_cancer_diagnosis_code',
                                  'metastatic_cancer_diagnosis_code',
                                  'payer_type', 'patient_state',
                                  'patient_race']),
                                ('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', RobustScaler())]),
                                 ['patient_age', 'labor_force_participation',
                                  'education_bachelors', 'family_dual_income',
                                  'education_college_or_above'])])

one-hot - wrapper


...

ColumnTransformer(transformers=[('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='UNKNOWN',
                                                                strategy='constant'))]),
                                 ['breast_cancer_diagnosis_code',
                                  'metastatic_cancer_diagnosis_code',
                                  'payer_type', 'patient_state',
                                  'patient_race']),
                                ('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', RobustScaler())]),
                                 ['patient_age', 'labor_force_participation',
                                  'education_bachelors', 'family_dual_income',
                                  'education_college_or_above'])])

unmodified - wrapper


ColumnTransformer(transformers=[('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='UNKNOWN',
                                                                strategy='constant'))]),
                                 ['breast_cancer_diagnosis_code',
                                  'metastatic_cancer_diagnosis_code',
                                  'patient_race', 'payer_type']),
                                ('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', RobustScaler())]),
                                 ['patient_age', 'bmi', 'commute_time',
                                  'race_native', 'education_stem_degree',
                                  'age_40s'])])

unmodified - no_selection


ColumnTransformer(transformers=[('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='UNKNOWN',
                                                                strategy='constant'))]),
                                 ['patient_race', 'payer_type', 'patient_state',
                                  'patient_zip3', 'Region', 'Division',
                                  'patient_gender',
                                  'breast_cancer_diagnosis_code',
                                  'breast_cancer_diagnosis_desc',
                                  'metastatic_cancer_diagnosis_code',
                                  'metastatic_first_novel_treatme...
                                  'age_60s', 'age_70s', 'age_over_80', 'male',
                                  'female', 'married', 'divorced',
                                  'never_married', 'widowed', 'family_size',
                                  'family_dual_income',
                                  'income_household_median',
                                  'income_household_under_5',
                                  'income_household_5_to_10',
                                  'income_household_10_to_15',
                                  'income_household_15_to_20',
                                  'income_household_20_to_25',
                                  'income_household_25_to_35',
                                  'income_household_35_to_50', ...])])

---

<a id="section4"></a>

# 4. Selección de modelos de regresión e hiperparámetros

En el apartado anterior se realizó la **selección de atributos** - proponiendo varios posibles subconjuntos de atributos para estudiar durante la experimentación - y se definió el **preprocesamiento** a realizar sobre estos atributos, de cara a prepararlos para el estudio.

El siguiente paso en el proceso de ciencia de datos, por tanto, sería el **entrenamiento y evaluación de modelos** a partir de los datos procesados y seleccionados. Ahora bien, antes de comenzar con el entrenamiento de los modelos, es necesario realizar una **selección de los modelos** que van a ser estudiados - junto a una **selección de hiperparámetros** a ser estudiado para cada modelo.

Por tanto, el objetivo de esta sección es:
1. Definir y estudiar los **estimadores** que se entrenaran y evaluarán durante el proceso de ciencia de datos.
    - Se propondrá una serie de modelos que servirán como *baseline* - el rendimiento base al que todos los modelos deberían aspirar a cumplir.
        - Sobre esto, se propondrán diversos modelos de regresión de familias típicas para los problemas de regresión - modelos **lineales** y **basados en árboles**.
    - Finalmente, se utilizarán ***ensembles*** - modelos estado del arte basados en agrupaciones de modelos más sencillos.
2. Seleccionar los **hiperparámetros** a estudiar para cada modelo.
    - Para ajustar el rendimiento de cada modelo al problema concreto, durante la experimentación se entrenará cada modelo varias veces - realizando una **validación cruzada** - probando distintos **conjuntos de hiperparámetros**.
    - Dependiendo de la complejidad en los hiperparámetros de cada modelo, la **exploración de los posibles hiperparámetros** se realizará de forma distinta:
        - `GridSearchCV`: Para modelos más simples, se realiza una **búsqueda exhaustiva** de todas las posibles combinaciones de hiperparámetros.
        - `BayesSearchCV`: Para modelos más complejos y lentos - donde el estudio exhaustivo no es factible - se realizará una **búsqueda probabilística** guiada.

En la siguiente sección de la libreta se realizará la **experimentación** sobre los modelos e hiperparámetros seleccionados. Para simplificar la **automatización** de estos experimentos, se almacena toda la información en un diccionario - `model_pipelines` -, utilizando la siguiente estructura:

```
model_pipelines{
    <nombre del modelo>: {
        "hyperparameter_search": "grid" | "random" | "bayes",
        "hyperparameter_grid: <hiperparámetros del modelo> | None,
        "models": {
            <subconjunto de datos>: <Pipeline>
        }
    }
    ...
}
```

Como se puede observar, para cada **modelo** se registra la siguiente información:
- `hyperparameter_search`: Tipo de busqueda de hiperparámetros que se realiza:
    - `"grid"`: Búsqueda exhaustiva de hiperparámetros - para modelos rápidos con pocos hiperparámetros.
    - `"random"`: Búsqueda completamente aleatoria - para modelos *simples* pero con **gran cantidad de hiperparámetros**.
    - `"bayes"`: Búsqueda aleatoria guiada con un modelo probabilístico subyacente - por el sobrecoste incluido, util principalmente para modelos **costosos** con **gran cantidad de hiperparámetros**.
- `hyperparameter_grid`: Diccionario incluyendo los hiperparámetros a buscar durante la experimentación.
- `models`: Para cada **posible subconjunto de atributos**, el *Pipeline* completo incluyendo preprocesamiento y modelo.

In [17]:
model_pipelines = {}


---

<a id="section4-1"></a>

## 4.1. *Baselines* - estimadores lineales, árboles y SVMs

En esta sección se definen los estimadores **simples** a utilizar durante la experimentación. Es importante distinguir que entendemos como **estimador simple / base** a cualquier estimador que **funciona de forma independiente** - frente a los *ensembles*, que funcionan como agrupaciones de estimadores simples. 

*(NOTA: Si bien la mayoría de modelos son a su vez sencillos a nivel de coste computacional, hay algunos (como las máquinas de vectores de soporte) que pueden tener costes elevados durante el entrenamiento. El abanico de posibles es más amplio del propuesto - se ha realizado una selección teniendo en cuenta el **tamaño del conjunto de datos** y la **alta dimensionalidad de este**.)*

En concreto, se van a estudiar algoritmos de las siguientes familias de modelos:
- **Lineales** - regresión lineal y algoritmos basados en ésta.
- **Arboles de decisión**.
- **Máquinas de vectores de soporte**.

El objetivo de los modelos propuestos en esta sección es servir como un *baseline* - una **puntuación base** que debería ser superada por el resto de modelos complejos, pero que sirve como un punto de referencia del **error promedio esperable si se entrenase el modelo más simple posible**.

---

### 4.1.1. Regresión lineal y variantes

Los modelos de **regresión lineal** buscan representar la dependencia entre un **atributo** y la **variable objetivo** a través de una **linea recta**, elegida con el fin de minimizar la distancia entre esta y todas las instancias del conjunto de datos - **reducir el error** ajustando los parámetros de la recta, buscando minimizar la **suma residual de errores cuadrados** en un proceso de optimización conocido como **mínimos cuadrados**.

Si bien el algoritmo inicial está propuesto para un único atributo, es posible **generalizarlo a múltiples atributos** - transformando la linea en un hiperplano, pero manteniendo el resto del proceso de optimización.

Estos modelos son **simples y muy eficientes** a la hora de ser entrenados y utilizados durante la inferencia. Ahora bien, también presentan el problema de que esta simpleza puede causar una **falta de capacidad computacional** a la hora de aprender relaciones más complejas entre datos, y **peor rendimiento cuando no existe independencia entre los atributos** - aunque existen diversas propuestas para solventar estos problemas.

En general - debido a la simplicidad de los modelos - se podrá realizar una **búsqueda exhaustiva de hiperparámetros** en un tiempo razonable.

#### - Regresión lineal

El modelo más simple de regresión lineal es el descrito previamente, implementado en `scikit-learn` en la clase `LinearRegression`. 

Debido a la simplicidad de este modelo, **no contiene ningún hiperparámetro a ajustar**. Ahora bien, durante el entrenamiento se realizará un ajuste sobre la **agrupación de los valores de las variables categóricas** - por lo que se seguirá utilizando una *validación cruzada de rejilla*.

In [18]:
# LINEAR REGRESSION
MODEL_NAME = "linear_regression"
# Base linear regression has no hyperparameters to adjust
HYPERPARAMETER_SEARCH = "grid"
# Linear regression is not able to handle categorical attributes - so they must be codified as numerical attributes
CATEGORICAL_ENCODING = "one-hot"

# No hyperparameters
model_pipelines[MODEL_NAME] = {
    "hyperparameter_search": HYPERPARAMETER_SEARCH,
    "hyperparameter_grid": {},
    "categorical_encoding": CATEGORICAL_ENCODING,
    "models": {}
}

# For each subset of features to test, create a pipeline
for subset, preprocessing in preprocessing_pipelines[CATEGORICAL_ENCODING].items():

    pipeline = Pipeline([
        ("preprocessing", preprocessing),
        ("regression", LinearRegression())
    ])

    # Store the pipeline in the appropriate position
    model_pipelines[MODEL_NAME]["models"][subset] = pipeline

#### - Ridge (L2)

**Ridge**, también conocido como **regresión lineal L2**, es una variante del modelo original de regresión lineal que añade un **factor de penalización $\lambda$** al error a optimizar, con el fin de **reducir la complejidad final del modelo** - entendiendo la complejidad como **la media del valor cuadrado de los coeficientes** - penalizando los atributos con parámetros excesivamente altos.

De esta forma, se obtienen modelos con **coeficientes más pequeños** a mayores valores de $\alpha$ - llevando a un modelo **más robusto ante variables correlacionadas** y **más resistente al sobreajuste**. Ahora bien, el modelo en general **no reduce el número de atributos utilizados**.

El modelo (`Ridge` en `scikit-learn`) tiene los siguientes **hiperparámetros** a ajustar:
- **Alfa ($\alpha$)**: Constante que regula el **factor de penalización** a los parámetros. Valores más altos indican **penalizaciones más altas**.

In [19]:
# LINEAR REGRESSION - L2
MODEL_NAME = "ridge_l2"
# Due to the simplicity, an exhaustive parameter search will finish in a reasonable time
HYPERPARAMETER_SEARCH = "grid"
# Linear regression is not able to handle categorical attributes - so they must be codified as numerical attributes
CATEGORICAL_ENCODING = "one-hot"
HYPERPARAMETER_GRID = {
    "regression__alpha": [1*10**num for num in range(-6, 7)]
}

model_pipelines[MODEL_NAME] = {
    "hyperparameter_search": HYPERPARAMETER_SEARCH,
    "hyperparameter_grid": HYPERPARAMETER_GRID,
    "categorical_encoding": CATEGORICAL_ENCODING,
    "models": {}
}

# For each subset of features to test, create a pipeline
for subset, preprocessing in preprocessing_pipelines[CATEGORICAL_ENCODING].items():

    pipeline = Pipeline([
        ("preprocessing", preprocessing),
        ("regression", Ridge(random_state=RANDOM_SEED))
    ])

    # Store the pipeline in the appropriate position
    model_pipelines[MODEL_NAME]["models"][subset] = pipeline

#### - Lasso (L1)

***Lasso (Least Absolute Shrunkage and Selection Operator)***, también conocido como **regresión lineal L1**, es una variante del modelo original de regresión lineal que añade un **factor de penalización $\lambda$** al error a optimizar, buscando reducir la **complejidad del modelo** - de forma muy similar a *Ridge*.

La diferencia principal entre **Ridge** y **Lasso** reside en como se calcula esta complejidad - pasando en este caso a medirla como la **media del valor absoluto de los coeficientes** frente al valor cuadrado. Esto se traduce en un modelo que **filtra los atributos menos relevantes** reduciendo sus coeficientes a 0 - pero más sensible a las **variables correlacionadas**.

El modelo (`Lasso` en `scikit-learn`) tiene los siguientes **hiperparámetros** a ajustar:
- **Alfa ($\alpha$)**: Constante que regula el **factor de penalización** a los parámetros. Valores más altos indican **penalizaciones más altas**.

In [20]:
# LINEAR REGRESSION - L1
MODEL_NAME = "lasso_l1"
# Due to the simplicity, an exhaustive parameter search will finish in a reasonable time
HYPERPARAMETER_SEARCH = "grid"
# Linear regression is not able to handle categorical attributes - so they must be codified as numerical attributes
CATEGORICAL_ENCODING = "one-hot"
HYPERPARAMETER_GRID = {
    "regression__alpha": [1*10**num for num in range(-6, 7)]
}

model_pipelines[MODEL_NAME] = {
    "hyperparameter_search": HYPERPARAMETER_SEARCH,
    "hyperparameter_grid": HYPERPARAMETER_GRID,
    "categorical_encoding": CATEGORICAL_ENCODING,
    "models": {}
}

# For each subset of features to test, create a pipeline
for subset, preprocessing in preprocessing_pipelines[CATEGORICAL_ENCODING].items():

    pipeline = Pipeline([
        ("preprocessing", preprocessing),
        ("regression", Lasso(random_state=RANDOM_SEED))
    ])

    # Store the pipeline in the appropriate position
    model_pipelines[MODEL_NAME]["models"][subset] = pipeline

#### - *Elastic-Net* (L1 y L2)

**Elastic-Net** es una combinación de los dos modelos de regresión lineal descritos previamente - **Ridge** (L1) y **Lasso** (L2) - donde se aplican a la vez ambas penalizaciones a la complejidad del modelo de forma **proporcional a un ratio `r`** determinado como hiperparámetro.

De esta forma, el modelo es capaz de aprovechar el comportamiento de ambas aproximaciones - modelos **más resistentes al sobreajuste y a las variables correlacionadas** (Lasso) pero capaces de **filtrar variables irrelevantes** (Ridge).

El modelo (`ElasticNet` en `scikit-learn`) tiene los siguientes **hiperparámetros** a ajustar:
- **Alfa ($\alpha$)**: Constante que regula el **factor de penalización** a los parámetros. Valores más altos indican **penalizaciones más altas**.
- **Ratio (`l1_ratio`)**: Valor en el rango $[0, 1]$ que determina **el ratio en el que se aplica la penalización de Ridge**. Concretamente:
    - `0` significa un modelo **Lasso (L2)**.
    - `1` significa un modelo **Ridge (L1)**.
    - Cualquier otro número significa una combinación, donde **números mayores** indican mayor influencia de Ridge, y **números menores** indican mayor influencia de Lasso.

In [106]:
# LINEAR REGRESSION - L1
MODEL_NAME = "elastic_net"
# Due to the simplicity, an exhaustive parameter search will finish in a reasonable time
HYPERPARAMETER_SEARCH = "grid"
# Linear regression is not able to handle categorical attributes - so they must be codified as numerical attributes
CATEGORICAL_ENCODING = "one-hot"
HYPERPARAMETER_GRID = {
    "regression__alpha": [1*10**num for num in range(-6, 7)],
    "regression__l1_ratio": [0.25, 0.5, 0.75]
}

model_pipelines[MODEL_NAME] = {
    "hyperparameter_search": HYPERPARAMETER_SEARCH,
    "hyperparameter_grid": HYPERPARAMETER_GRID,
    "categorical_encoding": CATEGORICAL_ENCODING,
    "models": {}
}

# For each subset of features to test, create a pipeline
for subset, preprocessing in preprocessing_pipelines[CATEGORICAL_ENCODING].items():

    pipeline = Pipeline([
        ("preprocessing", preprocessing),
        ("regression", ElasticNet(random_state=RANDOM_SEED))
    ])

    # Store the pipeline in the appropriate position
    model_pipelines[MODEL_NAME]["models"][subset] = pipeline

---

### 4.1.2. Árboles de decisión

Los modelos de **árboles de decisiones** buscan representar el conocimiento aprendido por el modelo a través de una **estructura de reglas jerárquica en forma de arbol** dividida en **nodos**, **ramas** y **hojas**; donde:
- **Nodo**: Un nodo interno del arbol - no final - donde se **comprueba el valor de un atributo**. El nodo se **ramifica** en tantas ramas como posibles resultados tenga la comprobación.
- **Rama**: Tras la comprobación, cada **rama** representa un posible resultado de la comprobación, generalmente un **posible valor del atributo** - para atributos categóricos - o **si el atributo es mayor o menos a un umbral** - para atributos numéricos. Esta rama puede llevar a otro nodo, donde se repetiría el proceso, o a una **hoja**.
- **Hoja**: Un **valor final predicho** para la variable objetivo - el valor estimado para una combinación de **atributos y sus valores**, representados por los nodos y ramas tomados para llegar hasta la hoja.

Por tanto, el objetivo de un algoritmo de aprendizaje de árboles de decisión es aprender el **conjunto de reglas** - concretamente, los **nodos**, **ramificaciones** y **hojas** que contiene el arbol - para representar de la forma más precisa posible el conjunto de datos.

En general, los árboles de decisión son modelos muy utilizadospor su **simpleza a la hora de interpretarlos** y a su **facilidad** para ser aplicados a diversos problemas. Ahora bien, **pueden conducir al sobreajuste** si no se eligen con cuidado los hiperparámetros y son **inestables** ante variaciones pequeñas en el conjunto de datos. 

Por esto, generalmente suelen ser utilizados como modelos más sencillos dentro de **ensembles** - como se verá en el siguiente apartado.

---

La implementación de los árboles de decisión para regresión en `scikit-learn` se encuentra en la clase `DecisionTreeRegressor`. Esta clase toma los siguientes **hiperparámetros**:
- **Profundidad máxima (`max_depth`)**: Profundidad máxima del arbol. A mayor profundidad, más complejas pueden ser las divisiones aprendidas - pero más propenso es a sobreajustar el modelo.
- **Número mínimo de instancias para particionar un nodo (`min_samples_split`)**: Para poder particionar un nodo - y que no sea una hoja -, este nodo debe tener al menos el número de instancias indicado.
- **Número mínimo de instancias por hoja (`min_samples_leaf`)**: Un nodo nunca se particiona si las hojas resultantes tienen menos instancias de las especificadas.
- **Criterio de particionamiento (`criterion`)**: Las particiones de los nodos se seleccionan buscando la **partición que minimiza el error resultante**. Las opciones a considerar son:
    - `squared_error`: Error cuadrático medio.
    - `friedman_mse`: Error absoluto medio utilizando una **corrección de Friedman** para considerar además las **probabilidades** de las particiones resultantes.
    - `absolute_error`: Error absoluto medio.

Debido al número elevado de hiperparámetros de hiperparámetros por estimar, una **búsqueda exhaustiva** de todas las posibles combinaciones no resulta exhaustiva. Por esto, se optará por explorar los hiperparámetros de forma **aleatoria**.

In [22]:
# DECISION TREE
MODEL_NAME = "decision_tree"
# Due to the high number of parameters, a randomized search is necessary
HYPERPARAMETER_SEARCH = "random"
# Decision trees do not handle categorical attributes natively - so encoding is necessary
CATEGORICAL_ENCODING = "one-hot"
HYPERPARAMETER_GRID = {
    "regression__max_depth": Integer(1, 20),
    "regression__min_samples_split": Integer(2, 50),
    "regression__min_samples_leaf": Integer(1, 50),
    "regression__criterion": Categorical(["squared_error", "friedman_mse", "absolute_error"])
}

model_pipelines[MODEL_NAME] = {
    "hyperparameter_search": HYPERPARAMETER_SEARCH,
    "hyperparameter_grid": HYPERPARAMETER_GRID,
    "categorical_encoding": CATEGORICAL_ENCODING,
    "models": {}
}

# For each subset of features to test, create a pipeline
for subset, preprocessing in preprocessing_pipelines[CATEGORICAL_ENCODING].items():

    pipeline = Pipeline([
        ("preprocessing", preprocessing),
        ("regression", DecisionTreeRegressor(random_state=RANDOM_SEED))
    ])

    # Store the pipeline in the appropriate position
    model_pipelines[MODEL_NAME]["models"][subset] = pipeline

---

### 4.1.3. Máquinas de vectores de soporte

Los modelos de **máquina de vector de soporte**, de forma similar a los modelos de regresión lineal, buscan encontrar un **hiperplano** que sea capaz de dividir el conjunto de datos de forma óptima. Ahora bien, en este caso se busca que el hiperplano tenga un **márgen respecto a los vectores de soporte** - las instancias más cercanas al hiperplano - cuya funcionalidad depende del tipo de problema a resolver:
- **Clasificación**: El caso más típico, se busca un **márgen máximo** para distinguir las clases del conjunto de datos - se busca el hiperplano **más genérico** capaz de distinguir las instancias del conjunto de datos.
- **Regresión**: En este caso, el margen cumple la idea contraria - las instancias **dentro del margen** no son relevantes para el modelo, al considerarse que son suficientemente cercanas a la predicción del modelo. En este caso, se busca encontrar el hiperplano que **minimice la distancia de todas las instancias a su margen** - estableciendo este margen en base a un **error mínimo $\epsilon$**.

La otra gran particularidad de estos modelos son los **métodos kernel**. En ocasiones, no es posible encontrar un hiperplano que sea capaz de cumplir las restricciones impuestas - o dividir de forma limpia el conjunto de datos o minimizar la distancia a las instancias. En estos casos, las máquinas de vector soporte son capaces de **proyectar las instancias del conjunto de datos en dimensionalidades más altas** a traves de lo que se conocen como **funciones Kernel** - buscando, de esta forma, un hiperplano de mayor dimensionalidad capaz de cumplir las condiciones necesarias.

En general, las máquinas de vectores de soporte son modelos **efectivos en problemas de alta dimensionalidad** y **eficientes con los datos** - al centrarse principalmente en los vectores de soporte que definen los márgenes del hiperplano. Ahora bien, son modelos **lentos de entrenar debido a su complejidad** y **propensos al sobreajuste** si no se cuidan sus hiperparámetros.

---

Existen varias implementaciones de estos modelos en `scikit-learn`. En concreto, se van a considerar **dos**:

#### - `LinearSVR`

La clase `LinearSVR` implementa una máquina de vectores de soporte para regresión optimizada para utilizar una **función kernel lineal** - una implementación más rápida pero limitada únicamente a este tipo de Kernel.

Esta clase toma los siguientes hiperparámetros:
- **Epsilon (`epsilon`):** Margen de error del hiperplano. En general, las instancias que se encuentran a menos de $\epsilon$ del hiperplano **se ignoran a la hora de optimizar el error**.
- **Tolerancia (`tol`)**: Tolerancia durante el entrenamiento. Una vez el error sea menor a la tolerancia, el proceso de entrenamiento se detiene.
- **Parametro de regularización (`C`)**: La potencia de la regularización realizada por el modelo es **inversamente proporcional** al valor de `C`.

Las máquinas de vectores de soporte son algoritmos **lentos de entrenar**. Por tanto - junto al **número elevado de hiperparámetros** -, se opta por una **búsqueda aleatoria de hiperparámetros**.

In [23]:
# SUPPORT VECTOR MACHINE - LINEAR KERNEL
MODEL_NAME = "svm_linear"
# Due to the high number of parameters, a randomized search is necessary
HYPERPARAMETER_SEARCH = "random"
# Support-vector machines do not handle categorical attributes natively - so encoding is necessary
CATEGORICAL_ENCODING = "one-hot"
HYPERPARAMETER_GRID = {
    "regression__epsilon": Real(0, 10),
    "regression__tol": Real(1e-6, 1e-3),
    "regression__C": Real(0.1, 100)
}

model_pipelines[MODEL_NAME] = {
    "hyperparameter_search": HYPERPARAMETER_SEARCH,
    "hyperparameter_grid": HYPERPARAMETER_GRID,
    "categorical_encoding": CATEGORICAL_ENCODING,
    "models": {}
}

# For each subset of features to test, create a pipeline
for subset, preprocessing in preprocessing_pipelines[CATEGORICAL_ENCODING].items():

    pipeline = Pipeline([
        ("preprocessing", preprocessing),
        ("regression", LinearSVR(random_state=RANDOM_SEED))
    ])

    # Store the pipeline in the appropriate position
    model_pipelines[MODEL_NAME]["models"][subset] = pipeline

#### - `SVR`

La clase `SVR` implementa una máquina de vectores de soporte para regresión general - capaz de utilizar **diversas funciones kernel no lineares** a costa de un coste computacional más elevado.

Esta clase toma los siguientes hiperparámetros:
- **Función kernel (`kernel`):** Función kernel a utilizar por el algoritmo. En concreto, se van a considerar las siguientes:
    - `poly`: Función **polinómica**. Contiene un hiperparámetro adicional - `degree`, el **grado del polinomio**.
    - `rbf`: Función **gaussiana**.
    - `sigmoid`: Función **sigmoide**.
- **Epsilon (`epsilon`):** Margen de error del hiperplano. En general, las instancias que se encuentran a menos de $\epsilon$ del hiperplano **se ignoran a la hora de optimizar el error**.
- **Tolerancia (`tol`)**: Tolerancia durante el entrenamiento. Una vez el error sea menor a la tolerancia, el proceso de entrenamiento se detiene.
- **Parametro de regularización (`C`)**: La potencia de la regularización realizada por el modelo es **inversamente proporcional** al valor de `C`.

Debido a los hiperparámetros distintos utilizados por cada función kernel, se proponen **tres modelos separados** - uno para cada función. Todos ellos se ajustan a través de **búsquedas aleatorias de hiperparámetros**.

##### - *SVR polinómica*

In [24]:
# SUPPORT VECTOR MACHINE - POLYNOMIC KERNEL
MODEL_NAME = "svm_poly"
# Due to the high number of parameters, a randomized search is necessary
HYPERPARAMETER_SEARCH = "random"
# Support-vector machines do not handle categorical attributes natively - so encoding is necessary
CATEGORICAL_ENCODING = "one-hot"
HYPERPARAMETER_GRID = {
    "regression__degree": Integer(2, 5),
    "regression__epsilon": Real(0, 10),
    "regression__tol": Real(1e-6, 1e-3),
    "regression__C": Real(0.1, 100)
}

model_pipelines[MODEL_NAME] = {
    "hyperparameter_search": HYPERPARAMETER_SEARCH,
    "hyperparameter_grid": HYPERPARAMETER_GRID,
    "categorical_encoding": CATEGORICAL_ENCODING,
    "models": {}
}

# For each subset of features to test, create a pipeline
for subset, preprocessing in preprocessing_pipelines[CATEGORICAL_ENCODING].items():

    pipeline = Pipeline([
        ("preprocessing", preprocessing),
        ("regression", SVR(kernel="poly"))
    ])

    # Store the pipeline in the appropriate position
    model_pipelines[MODEL_NAME]["models"][subset] = pipeline

##### - *SVR gaussiana*

In [25]:
# SUPPORT VECTOR MACHINE - GAUSSIAN KERNEL
MODEL_NAME = "svm_rbf"
# Due to the high number of parameters, a randomized search is necessary
HYPERPARAMETER_SEARCH = "random"
# Support-vector machines do not handle categorical attributes natively - so encoding is necessary
CATEGORICAL_ENCODING = "one-hot"
HYPERPARAMETER_GRID = {
    "regression__epsilon": Real(0, 10),
    "regression__tol": Real(1e-6, 1e-3),
    "regression__C": Real(0.1, 100)
}

model_pipelines[MODEL_NAME] = {
    "hyperparameter_search": HYPERPARAMETER_SEARCH,
    "hyperparameter_grid": HYPERPARAMETER_GRID,
    "categorical_encoding": CATEGORICAL_ENCODING,
    "models": {}
}

# For each subset of features to test, create a pipeline
for subset, preprocessing in preprocessing_pipelines[CATEGORICAL_ENCODING].items():

    pipeline = Pipeline([
        ("preprocessing", preprocessing),
        ("regression", SVR(kernel="rbf"))
    ])

    # Store the pipeline in the appropriate position
    model_pipelines[MODEL_NAME]["models"][subset] = pipeline

##### - *SVR sigmoide*

In [26]:
# SUPPORT VECTOR MACHINE - SIGMOID KERNEL
MODEL_NAME = "svm_sigmoid"
# Due to the high number of parameters, a randomized search is necessary
HYPERPARAMETER_SEARCH = "random"
# Support-vector machines do not handle categorical attributes natively - so encoding is necessary
CATEGORICAL_ENCODING = "one-hot"
HYPERPARAMETER_GRID = {
    "regression__epsilon": Real(0, 10),
    "regression__tol": Real(1e-6, 1e-3),
    "regression__C": Real(0.1, 100)
}

model_pipelines[MODEL_NAME] = {
    "hyperparameter_search": HYPERPARAMETER_SEARCH,
    "hyperparameter_grid": HYPERPARAMETER_GRID,
    "categorical_encoding": CATEGORICAL_ENCODING,
    "models": {}
}

# For each subset of features to test, create a pipeline
for subset, preprocessing in preprocessing_pipelines[CATEGORICAL_ENCODING].items():

    pipeline = Pipeline([
        ("preprocessing", preprocessing),
        ("regression", SVR(kernel="sigmoid"))
    ])

    # Store the pipeline in the appropriate position
    model_pipelines[MODEL_NAME]["models"][subset] = pipeline

---

<a id="section4-2"></a>

## 4.2. *Ensembles* - Estimadores basados en agrupaciones de estimadores simples

En esta sección se definen los *ensembles* a utilizar durante la experimentación. 

Un ***ensemble*** es un algoritmo de aprendizaje automático que utiliza una **agrupación de modelos sencillos** - generalmente **con menor complejidad** que si se utilizasen de forma independiente - para su funcionamiento. Estos modelos son entrenados sobre el conjunto de datos - frecuentemente sobre **subconjuntos aleatorios de instancias y atributos** -, y el resultado del *ensemble* se obtiene como una **agrupación** de los resultados de todos los modelos - generalmente mediante algún tipo de **ponderación**.

Dependiendo de cómo se utilizan los modelos simples, se pueden identificar dos grandes familias de *ensembles*:
- **Bagging (Boostrap Aggregating)**: Cada modelo se entrena de forma **independiente** al resto.
- **Boosting**: Los modelos se entrenan de forma **secuencial** - donde el error de un modelo afecta a cómo se entrena el siguiente modelo.

Ahora bien, los modelos de *ensembles* tienen algunos detrimentos compartidos por todas las familias:
- El **coste computacional** del entrenamiento es considerablemente superior al de los modelos simples - al ser, esencialmente, **entrenamientos repetidos de grandes cantidades de modelos simples**.
- Para maximizar el rendimiento del *ensemble*, es necesario realizar un **ajuste de un gran número de hiperparámetros** - haciendo las **búsquedas exhaustivas** imposibles, y teniendo que recurrir a búsquedas aleatorias para tiempos razonables de entrenamiento.

Los modelos de *ensemble*, junto a modelos de **aprendizaje profundo** - como las **redes neuronales** - son considerados el **estado del arte** del aprendizaje supervisado actualmente. Es de esperar que sus resultados superen al resultado de los modelos de *baseline*.

---

### 4.2.1. *Bagging (Boostrap Aggregating)*

Los modelos de **bagging (bootstrap aggregating)** son agrupaciones de modelos de aprendizaje automático simples, donde cada modelo se entrena sobre **un subconjunto aleatorio de instancias del conjunto de datos** mediante un proceso conocido como **bootstrapping** (un muestreo aleatorio uniforme con reemplazo). El resultado final del *ensemble* consiste en la **unión** de los resultados de cada uno de los modelos subyacentes - ya sea por **votación**, eligiendo el resultado mayoritario, o por **promedio** de los resultados.

La principal meta de esta técnica es **reducir la varianza y el sobreajuste** de los modelos simples subyacentes - al estar cada modelo expuesto a un subconjunto aleatorio de datos con posibles duplicados, el conjunto de los modelos entrenados va a tener una mayor **diversidad**, permitiendo al *ensemble* ser capaz de **generalizar** de forma más robusta.

En general, las técnicas de *bagging* obtienen modelos más **robustos** debido a su resistencia al sobreajuste. Además, son facilmente **paralelizables** - al entrenarse de forma independiente cada modelo - y **compatibles con cualquier modelo subyacente**, aunque funcionan mejor al utilizarse con modelos propensos a sobreajustar como los **árboles de decisión**.

Se van a probar concretamente **dos** modelos de *bagging* usados con frecuencia en la actualidad - si bien sería posible crear un *ensemble* con cualquier modelo.

#### - *Random Forests*

***Random Forest*** es un modelo de aprendizaje automático basado en *ensembles* de **bagging**, consistente en un conjunto de **arboles de decisión profundos** donde cada arbol se entrena a la vez sobre un **subconjunto aleatorio de datos** y un **subconjunto aleatorio de los atributos** - siendo los atributos aleatorios la principal diferencia con el resto de modelos de *bagging*.

Al introducir estas dos fuentes de aleatoriedad, cada arbol **sobreajusta** a una parte distinta del conjunto de datos - teniendo, en conjunto, un grupo de **arboles de decisión profundos especializados** capaces de obtener un resultado **más generalizado** a través de un promedio de todas las salidas.

---

El modelo está implementado en `scikit-learn` a través de la clase `RandomForestRegressor`. Si bien se pueden ajustar los hiperparámetros de igual manera que con los arboles de decisiones individualmente, se van a considerar los **siguientes hiperparámetros**:
- **Número de estimadores (`n_estimators`):** Número de árboles a considerar en el modelo.
- **Número de atributos considerados por arbol (`max_features`):** Dependiendo del valor, se considera el siguiente número de atributos por arbol:
    - `None`: Cada arbol tiene acceso a **todos los atributos**.
    - `sqrt`: Cada arbol tiene acceso a un subconjunto de tamaño equivalente a **la raiz cuadrada del número de atributos**.
    - **Valor numérico**: Cada arbol tiene acceso al **porcentaje indicado del número de atributos**. En general, **0.3** es un valor utilizado con frecuencia en la literatura.
- **Profundidad máxima (`max_depth`):** Profundidad máxima de cada arbol entrenado.
- **Número mínimo de instancias para particionar un nodo (`min_samples_split`)**: Para poder particionar un nodo - y que no sea una hoja -, este nodo debe tener al menos el número de instancias indicado.

In [27]:
# RANDOM FOREST
MODEL_NAME = "random_forest"
# Due to the high number of parameters, a randomized search is necessary
HYPERPARAMETER_SEARCH = "bayes"
# Random forests do not handle categorical attributes natively - so encoding is necessary
CATEGORICAL_ENCODING = "one-hot"
HYPERPARAMETER_GRID = {
    "regression__n_estimators": Integer(50, 200),
    "regression__max_features": Real(0, 1),
    "regression__max_depth": Integer(1, 50),
    "regression__min_samples_split": Integer(2, 50)
}

model_pipelines[MODEL_NAME] = {
    "hyperparameter_search": HYPERPARAMETER_SEARCH,
    "hyperparameter_grid": HYPERPARAMETER_GRID,
    "categorical_encoding": CATEGORICAL_ENCODING,
    "models": {}
}

# For each subset of features to test, create a pipeline
for subset, preprocessing in preprocessing_pipelines[CATEGORICAL_ENCODING].items():

    pipeline = Pipeline([
        ("preprocessing", preprocessing),
        ("regression", RandomForestRegressor(random_state=RANDOM_SEED))
    ])

    # Store the pipeline in the appropriate position
    model_pipelines[MODEL_NAME]["models"][subset] = pipeline

#### - *Extremely Randomized Trees (Extra Trees)*

***Extremely Randomized Trees*** (también conocido como ***Extra Trees***) es una variación del modelo de *Random Forest* que añade un tercer factor de aleatoriedad a la forma en la que se construyen los árboles.

Durante el proceso de construcción del arbol, en cada nodo se elige el atributo que **más reduce el error a la hora de particionar el conjunto de datos** - comprobando, para cada posible atributo, el **umbral de partición** que más reduce el error. En el caso de ***Extra Trees***, en vez de elegir el mejor umbral se elige **un umbral aleatorio para cada atributo** - eligiendo, después, el **atributo con umbral aleatorio** que más reduce el error.

De esta manera se **reduce más la varianza del modelo final** al tener árboles más aleatorios - a riesgo de **aumentar el sesgo final**.

---

El modelo está implementado en `scikit-learn` a través de la clase `ExtraTreesRegressor`. Los hiperparámetros a ajustar son, esencialmente, los mismos que se han ajustado en *Random Forest*:
- **Número de estimadores (`n_estimators`):** Número de árboles a considerar en el modelo.
- **Número de atributos considerados por arbol (`max_features`):** Dependiendo del valor, se considera el siguiente número de atributos por arbol:
    - `None`: Cada arbol tiene acceso a **todos los atributos**.
    - `sqrt`: Cada arbol tiene acceso a un subconjunto de tamaño equivalente a **la raiz cuadrada del número de atributos**.
    - **Valor numérico**: Cada arbol tiene acceso al **porcentaje indicado del número de atributos**. En general, **0.3** es un valor utilizado con frecuencia en la literatura.
- **Profundidad máxima (`max_depth`):** Profundidad máxima de cada arbol entrenado.
- **Número mínimo de instancias para particionar un nodo (`min_samples_split`)**: Para poder particionar un nodo - y que no sea una hoja -, este nodo debe tener al menos el número de instancias indicado.

In [28]:
# EXTREMELY RANDOM TREES (EXTRA TREES)
MODEL_NAME = "extra_trees"
# Due to the high training time, a randomized guided search is necessary
HYPERPARAMETER_SEARCH = "bayes"
# Extra trees do not handle categorical attributes natively - so encoding is necessary
CATEGORICAL_ENCODING = "one-hot"
HYPERPARAMETER_GRID = {
    "regression__n_estimators": Integer(50, 200),
    "regression__max_features": Real(0, 1),
    "regression__max_depth": Integer(1, 50),
    "regression__min_samples_split": Integer(2, 50)
}

model_pipelines[MODEL_NAME] = {
    "hyperparameter_search": HYPERPARAMETER_SEARCH,
    "hyperparameter_grid": HYPERPARAMETER_GRID,
    "categorical_encoding": CATEGORICAL_ENCODING,
    "models": {}
}

# For each subset of features to test, create a pipeline
for subset, preprocessing in preprocessing_pipelines[CATEGORICAL_ENCODING].items():

    pipeline = Pipeline([
        ("preprocessing", preprocessing),
        ("regression", ExtraTreesRegressor(random_state=RANDOM_SEED))
    ])

    # Store the pipeline in the appropriate position
    model_pipelines[MODEL_NAME]["models"][subset] = pipeline

---

### 4.2.2. *Boosting*

Los modelos de ***Boosting*** son agrupaciones de modelos de aprendizaje automático **debiles** (entendiendose como *debil* a un modelo con poca capacidad computacional), donde los modelos se entrenan de forma **secuencial** - de forma que los errores de un modelo son considerados en el siguiente modelo mediante **importancias asignadas a cada instancia del conjunto de datos**, teniendo las instancias con mayor error un mayor peso a la hora de entrenar los modelos. El resultado final del *ensemble* consiste en una **ponderación** de los resultados de todos los modelos - donde los modelos con menor error tienen más peso en la votación final.

A diferencia de **bagging** - donde cada modelo se entrena en paralelo sobre un subconjunto aleatorio de atributos -, **boosting** entrena los modelos sobre el conjunto de datos completo ponderado.

La principal meta de esta técnica es **crear un conjunto de datos capaz de identificar relaciones complejas en el conjunto de datos** - al entrenar cada modelo teniendo en cuenta los errores producidos por el modelo anterior, se acaban produciendo modelos **muy especializados** para errores específicos. Además, la **agrupación ponderada** de los resultados permite al *ensemble* obtener resultados **sustancialmente mejores** a los de los modelos individuales.

Las técnicas de **boosting** tienden a obtener **mejores resultados** tanto en varianza como en sesgo - aunque suelen ser modelos **más caros y lentos de entrenar** (al entrenarse de forma secuencial) y **más susceptibles a ruido y valores extremos** en el conjunto de datos.


---

Se considera **un modelo clásico de boosting** - actualmente en desuso a favor de otras técnicas que se verán posteriormente:

#### - *Adaptive Boosting (AdaBoost)*

***Adaptive Boosting*** (también conocido como ***AdaBoost***) es el modelo que introdujo el concepto de *boosting* y ha sido, durante muchos años, uno de los principales *ensembles* utilizados en el proceso de ciencia de datos.

*AdaBoost* entrena de forma secuencial **modelos débiles** - concretamente **tocones**, arboles de decisión de baja profundidad - sobre el **conjunto de datos completo**. Ahora bien, cada instancia del conjunto de datos tiene una **importancia** ponderada por su peso, de forma que:
- Tras cada iteración, el peso de las instancias se ajusta -  las **instancias predecidas con mayor error aumentan su importancia**, por lo que **las instancias más dificiles de predecir** tienen los pesos más elevados.
- Cuando se entrena un modelo, la importancia de cada instancia se pondera por su peso - por lo que **los modelos se sesgan hacia las instancias más dificiles de clasificar**.

Además, **cada modelo tiene un peso asociado** en base a su error total. El resultado final del modelo es una **agregación ponderada** de todos los resultados - donde los modelos con menor error tienen una mayor importancia en la ponderación final.

---

El modelo está implementado en `scikit-learn` en la clase `AdaBoostRegressor`. Si bien nos deja elegir cualquier modelo como estimador base, se utilizará el modelo por defecto (un **arbol de decisión de profundidad 3**). Se consideran los siguientes hiperparámetros:
- **Número de estimadores (`n_estimators`):** Número de árboles a considerar en el modelo.
- **Ratio de aprendizaje (`learning_rate`):** Ponderación aplicada a cada modelo nuevo - en general, representa la **velocidad** a la que el modelo puede aprender, donde **valores más altos** se traducen en cambios más rápidos y **valores más bajos** se traducen en cambios más lentos.

In [29]:
# ADAPTIVE BOOSTING (ADABOOST)
MODEL_NAME = "adaboost"
# Due to the high number of parameters, a randomized search is necessary
HYPERPARAMETER_SEARCH = "bayes"
# Adaboost does not handle categorical attributes natively - so encoding is necessary
CATEGORICAL_ENCODING = "one-hot"
HYPERPARAMETER_GRID = {
    "regression__n_estimators": Integer(50, 200),
    "regression__learning_rate": Real(1e-4, 10)
}

model_pipelines[MODEL_NAME] = {
    "hyperparameter_search": HYPERPARAMETER_SEARCH,
    "hyperparameter_grid": HYPERPARAMETER_GRID,
    "categorical_encoding": CATEGORICAL_ENCODING,
    "models": {}
}

# For each subset of features to test, create a pipeline
for subset, preprocessing in preprocessing_pipelines[CATEGORICAL_ENCODING].items():

    pipeline = Pipeline([
        ("preprocessing", preprocessing),
        ("regression", AdaBoostRegressor(random_state=RANDOM_SEED))
    ])

    # Store the pipeline in the appropriate position
    model_pipelines[MODEL_NAME]["models"][subset] = pipeline

---

### 4.2.3. *Gradient Boosting*

Los modelos de ***Gradient Boosting*** son variaciones de los modelos de **boosting** en los que:
- Se utiliza **gradiente descendiente** a la hora de calcular la importancia de las instancias del conjunto de datos - en vez de actualizar directamente los pesos en base a las **residuales**, se calcula el **gradiente** que maximiza la reducción de una **función de error genérica** para cada instancia - ponderando el entrenamiento del modelo nuevo utilizando este gradiente.
- Si bien se siguen usando modelos débiles, **se tiende a usar modelos más complejos que en las técnicas tradicionales**.
- Por lo general, las **implementaciones** de los modelos de *Gradient Boosting* ofrecen **soporte nativo a atributos categóricos** - sin que sea necesario codificar los atributos a través de *One-Hot Encoding*. 

Realmente, los modelos de *Gradient Boosting* son **generalizaciones** de los modelos originales de *boosting* (principalmente ***AdaBoost***) para poder utilizar **cualquier función de error genérica** - con la única restricción de necesitar una función **derivable** para poder usarse durante gradiente descendiente.

Por lo general, los modelos de *gradient boosting* ofrecen mejores resultados que las técnicas tradicionales de *boosting* - a cambio de un mayor coste computacional para su entrenamiento.

---

Se van a considerar **cuatro** modelos de *Gradient Boosting* - actualmente **estado del arte** para los problemas de aprendizaje automático sobre datos tabulares estructurados. En general, las diferencias entre estos modelos están más centradas en la **implementación técnica de los modelos** - aunque también existen algunas diferencias a la hora de definir los algoritmos:

#### - *Extreme Gradient Boosting (XGBoost)*

***eXtreme Gradient Boosting*** (también conocido como ***XGBoost***) es un modelo y una librería de código abierto creada con el objetivo de crear implementaciones **escalables, portables y capaces de ser distribuidas entre distintas máquinas**.

Concretamente, ***XGBoost*** es un algoritmo de *Gradient Boosting* que utiliza **árboles de decisión** como modelos base, con las siguientes características:
- En vez de utilizar **gradiente descendiente**, se utiliza el ***método de Newton-Raphson** para optimizar el error - utilizando **segundas derivadas** para calcular el gradiente que minimiza el error de los modelos.
- Los árboles se entrenan de forma **paralela** - pudiendo distribuir el entrenamiento entre varios computadores a la vez.

---

La implementación de ***XGBoost*** está disponible en la librería `xgboost` en la clase `XGBRegressor`, aunque es totalmente compatible con `scikit-learn` para preprocesamiento y experimentación. , según la [guía de la librería](https://xgboost.readthedocs.io/en/stable/tutorials/param_tuning.html#notes-on-parameter-tuning):
- **Número de estimadores (`n_estimators`):** Número de árboles / iteraciones a entrenar.
- **Profundidad máxima (`max_depth`):** Profundidad máxima de los árboles entrenados.
- **Ratio de aprendizaje (`learning_rate`):** Factor por el que se **multiplican las actualizaciones de pesos** para ralentizar el aprendizaje.
- **Umbral de mejora para partición (`gamma`)**: Es necesario que se reduzca el error al menos `gamma` para que se considere una partición.
- **Ratio de muestreo del conjunto de datos (`subsample`):** Porcentaje del conjunto de entrenamiento que se muestrea para entrenar cada arbol.
- **Ratio de muestreo de los atributos (`colsample_bytree`):** Porcentaje del conjunto de atributos que se muestrea para entrenar cada arbol.

In [30]:
# EXTREME GRADIENT BOOST (XGBOOST)
MODEL_NAME = "xgboost"
# Due to the high number of parameters, a randomized search is necessary
HYPERPARAMETER_SEARCH = "bayes"
# XGBoost handles categorical values automatically - so no encoding is required
CATEGORICAL_ENCODING = "unmodified"
HYPERPARAMETER_GRID = {
    "regression__n_estimators": Integer(50, 200),
    "regression__learning_rate": Real(0, 1),
    "regression__max_depth": Integer(4, 10),
    "regression__gamma": Real(0, 1e4),
    "regression__subsample": Real(0, 1),
    "regression__colsample_bytree": Real(0, 1)
}

model_pipelines[MODEL_NAME] = {
    "hyperparameter_search": HYPERPARAMETER_SEARCH,
    "hyperparameter_grid": HYPERPARAMETER_GRID,
    "categorical_encoding": CATEGORICAL_ENCODING,
    "models": {}
}

# For each subset of features to test, create a pipeline
for subset, preprocessing in preprocessing_pipelines[CATEGORICAL_ENCODING].items():

    pipeline = Pipeline([
        ("preprocessing", preprocessing),
        # XGBoost will attempt to use GPU if able to
        ("regression", XGBRegressor(random_state=RANDOM_SEED, device="gpu", enable_categorical=True))
    ])

    # Store the pipeline in the appropriate position
    model_pipelines[MODEL_NAME]["models"][subset] = pipeline

#### - *Categorical Boosting (CatBoost)*

***Categorical Boosting*** (también conocido como ***CatBoost***) es un modelo y una librería de código abierto creada por *Yandex* con el objetivo de ofrecer un modelo capaz de **trabajar directamente con atributos categóricos** sin necesidad de un preprocesamiento previo.

Concretamente, ***CatBoost*** es un algoritmo de *Gradient Boosting* que utiliza **árboles de decisión** como modelos base, con las siguientes características:
- ***Ordered Boosting***: Utilizar el mismo conjunto de datos para entrenar los árboles y calcular las residuales puede producir **sesgos** durante el entrenamiento. El algoritmo implementa un procedimiento de *boosting* ordenado - utilizando en cada iteración de *boosting* una **permutación aleatoria del conjunto de datos**, y considerando únicamente las **instancias ya vistas** a la hora de elegir los valores de la hoja - con el fin de paliar este problema.
- El algoritmo usa **árboles de decisiones *oblivious*** - árboles de decisión en los que, en cada profundidad **se usa el mismo umbral para todos los nodos**.
- El algoritmo es capaz de **trabajar directamente con atributos categóricos** - sin necesidad de pre-procesarlos previamente para transformarlos en atributos numéricos.

---

La implementación de ***CatBoost*** está disponible en la librería `catboost` en la clase `CatBoostRegressor`, aunque es totalmente compatible con `scikit-learn` para preprocesamiento y experimentación. Se consideran los siguientes parámetros, según la [guía de la librería](https://catboost.ai/docs/en/concepts/parameter-tuning):
- **Número de estimadores (`iterations`):** Número de árboles / iteraciones a entrenar.
- **Ratio de aprendizaje (`learning_rate`):** Factor por el que se **multiplican las actualizaciones de pesos** para ralentizar el aprendizaje.
- **Profundidad máxima (`max_depth`):** Profundidad máxima de los árboles entrenados.
- **Regularización del error (`l2_leaf_reg`):** Coeficiente utilizado para la regularización de tipo **Ridge** utilizada a la hora de calcular los errores.
- **Intensidad de la aleatoriedad (`random_strength`):** Multiplicador que se aplica a la **varianza** de cada posible partición, para introducir aleatoriedad durante el entrenamiento.

Existe una gran cantidad de posibles hiperparámetros a ajustar, por lo que se ha optado por utilizar un **subconjunto reducido** de estos.

In [31]:
# CATEGORICAL BOOSTING - CATBOOST
MODEL_NAME = "catboost"
# Due to the high number of parameters, a randomized search is necessary
HYPERPARAMETER_SEARCH = "bayes"
# CatBoost handles categorical values automatically - so no encoding is required
CATEGORICAL_ENCODING = "unmodified"
HYPERPARAMETER_GRID = {
    "regression__iterations": Integer(50, 200),
    "regression__learning_rate": Real(0, 1),
    "regression__max_depth": Integer(6, 10),
    "regression__l2_leaf_reg": Real(0, 10),
    "regression__random_strength": Real(1, 2)
}

model_pipelines[MODEL_NAME] = {
    "hyperparameter_search": HYPERPARAMETER_SEARCH,
    "hyperparameter_grid": HYPERPARAMETER_GRID,
    "categorical_encoding": CATEGORICAL_ENCODING,
    "models": {}
}

# For each subset of features to test, create a pipeline
for subset, preprocessing in preprocessing_pipelines[CATEGORICAL_ENCODING].items():

    pipeline = Pipeline([
        ("preprocessing", preprocessing),
        # CatBoost will attempt to use GPU if able to
        ("regression", CatBoostRegressor(random_state=RANDOM_SEED, task_type="gpu"))
    ])

    # Store the pipeline in the appropriate position
    model_pipelines[MODEL_NAME]["models"][subset] = pipeline

#### - *Light Gradient Boosting Machine (LGBM)*

***Light Gradient Boosting Machine*** (también conocido como ***LGBM***) es un modelo y una librería de código abierto creada por *Microsoft* con el objetivo de ofrecer *ensembles* **escalables y eficientes**.

Concretamente, ***LGBM*** es un algoritmo de *Gradient Boosting* que utiliza **árboles de decisión** como modelos base, con las siguientes características:
- **Crecimiento del arbol por hojas (Best-First)**: En vez de aumentar el tamaño de los árboles nivel a nivel - como la mayoría de algoritmos -, los árboles crecen **hoja a hoja** - se elige **expandir la hoja** que maximiza el gradiente que reduce el error.
- **Selección de umbrales basada en histogramas**: En vez de ordenar las instancias para buscar los umbrales de corte óptimos para cada atributo, *LGBM* en su lugar **discretiza los posibles umbrales de corte** - reduciendo de esta forma el posible número de umbrales a considerar.
- El algoritmo **trata de forma automática las variables categóricas** - transformándolas en un número reducido de variables numéricas.

---

La implementación de ***LGBM*** está disponible en la librería `lightgbm` en la clase `LGBMRegressor`, aunque es totalmente compatible con `scikit-learn` para preprocesamiento y experimentación. Se consideran los siguientes parámetros, según la [guía de la librería](https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html):
- **Profundidad máxima (`max_depth`):** Profundidad máxima a la que puede crecer cada arbol. Debido a cómo crecen los árboles, **un arbol que ha alcanzado su profundidad máxima puede seguir creciendo** mientras pueda seguir dividiendo hojas hasta la profundidad máxima.
- **Número máximo de hojas (`num_leaves`):** Número máximo de hojas que se pueden tener - independientemente de la profundidad.
- **Número mínimo de instancias por hoja (`min_data_in_leaf`):** Un nodo no se puede particionar si resulta en una hoja con un número de instancias asociadas menor a este valor.

In [32]:
# LIGHT GRADIENT BOOSTING MACHINE (LGBM)
MODEL_NAME = "lgbm"
# Due to the high number of parameters, a randomized search is necessary
HYPERPARAMETER_SEARCH = "bayes"
# LGBM handles categorical values automatically - so no encoding is required
CATEGORICAL_ENCODING = "unmodified"
HYPERPARAMETER_GRID = {
    "regression__max_depth": Integer(1, 10),
    "regression__num_leaves": Integer(10, 100),
    "regression__min_data_in_leaf": Integer(20, 200)
}

model_pipelines[MODEL_NAME] = {
    "hyperparameter_search": HYPERPARAMETER_SEARCH,
    "hyperparameter_grid": HYPERPARAMETER_GRID,
    "categorical_encoding": CATEGORICAL_ENCODING,
    "models": {}
}

# For each subset of features to test, create a pipeline
for subset, preprocessing in preprocessing_pipelines[CATEGORICAL_ENCODING].items():

    pipeline = Pipeline([
        ("preprocessing", preprocessing),
        # LGBM will attempt to use GPU if able to
        ("regression", LGBMRegressor(random_state=RANDOM_SEED, device="gpu"))
    ])

    # Store the pipeline in the appropriate position
    model_pipelines[MODEL_NAME]["models"][subset] = pipeline

#### - *Gradient Boosting* basado en histogramas (*HistGradientBoost*)

***Histogram Gradient Boosting*** (también conocido como ***HistGradientBoost***) es un modelo de *Gradient Boosting* ofrecido por `scikit-learn`, basado en el funcionamiento de *LightGBM* - y, específicamente, en el uso de **histogramas** para la elección de umbrales.

Concretamente, ***HistGradientBoost*** es un algoritmo de *Gradient Boosting* que utiliza **árboles de decisión** como modelos base, con las siguientes características:
- Los atributos numéricos son **discretizados en histogramas representados a través de números enteros** - permitiendo el uso de estructuras de datos basadas en valores enteros para agilizar la construcción de los árboles.
- Igual que el resto de algoritmos de *Gradient Boosting*, **los atributos categóricos se consideran directamente** - sin necesidad de realizar ningún tipo de codificación previo.

---

La implementación de ***HistGradientBoost*** está disponible directamente en la librería `scikit-learn` en la clase `HistGradientBoostingRegressor`. Se consideran los siguientes parámetros - esencialmente, **los mismos hiperparámetros que LightGBM**:
- **Profundidad máxima (`max_depth`):** Profundidad máxima a la que puede crecer cada arbol. Debido a cómo crecen los árboles, **un arbol que ha alcanzado su profundidad máxima puede seguir creciendo** mientras pueda seguir dividiendo hojas hasta la profundidad máxima.
- **Número máximo de hojas (`num_leaves`):** Número máximo de hojas que se pueden tener - independientemente de la profundidad.
- **Número mínimo de instancias por hoja (`min_samples_leaf`):** Un nodo no se puede particionar si resulta en una hoja con un número de instancias asociadas menor a este valor.
- **Porcentaje de atributos a considerar (`max_features`):** Utilizar un subconjunto reducido de atributos puede conducir a modelos con mayor capacidad de generalización.

In [33]:
# HISTOGRAM GRADIENT BOOSTING (HISTGRADIENTBOOST)
MODEL_NAME = "hstgradientboost"
# Due to the high number of parameters, a randomized search is necessary
HYPERPARAMETER_SEARCH = "bayes"
# LGBM handles categorical values automatically - so no encoding is required
CATEGORICAL_ENCODING = "unmodified"
HYPERPARAMETER_GRID = {
    "regression__max_depth": Integer(1, 10),
    "regression__num_leaves": Integer(10, 100),
    "regression__min_samples_leaf": Integer(20, 200),
    "regression__max_features": Real(0.3, 1)
}

model_pipelines[MODEL_NAME] = {
    "hyperparameter_search": HYPERPARAMETER_SEARCH,
    "hyperparameter_grid": HYPERPARAMETER_GRID,
    "categorical_encoding": CATEGORICAL_ENCODING,
    "models": {}
}

# For each subset of features to test, create a pipeline
for subset, preprocessing in preprocessing_pipelines[CATEGORICAL_ENCODING].items():

    pipeline = Pipeline([
        ("preprocessing", preprocessing),
        # HistGradientBoost is not compatible with GPU usage
        ("regression", HistGradientBoostingRegressor(categorical_features="from_dtype", random_state=RANDOM_SEED))
    ])

    # Store the pipeline in the appropriate position
    model_pipelines[MODEL_NAME]["models"][subset] = pipeline

---

Tras la declaración de **todos los modelos a utilizar durante la experimentación**, se muestran los contenidos de `model_pipelines`:

In [ ]:
for model_name in model_pipelines:
    print(f"Model: {model_name}")
    print(f"\tHyperparameter search type: {model_pipelines[model_name]['hyperparameter_search']}")
    print(f"\tCategorical encoding: {model_pipelines[model_name]['categorical_encoding']}")

Model: linear_regression
	Hyperparameter search type: grid
	Categorical encoding: one-hot
Model: ridge_l2
	Hyperparameter search type: grid
	Categorical encoding: one-hot
Model: lasso_l1
	Hyperparameter search type: grid
	Categorical encoding: one-hot
Model: elastic-net
	Hyperparameter search type: grid
	Categorical encoding: one-hot
Model: decision_tree
	Hyperparameter search type: random
	Categorical encoding: one-hot
Model: svm_linear
	Hyperparameter search type: random
	Categorical encoding: one-hot
Model: svm_poly
	Hyperparameter search type: random
	Categorical encoding: one-hot
Model: svm_rbf
	Hyperparameter search type: random
	Categorical encoding: one-hot
Model: svm_sigmoid
	Hyperparameter search type: random
	Categorical encoding: one-hot
Model: random_forest
	Hyperparameter search type: bayes
	Categorical encoding: one-hot
Model: extra_trees
	Hyperparameter search type: bayes
	Categorical encoding: one-hot
Model: adaboost
	Hyperparameter search type: bayes
	Categorical enco

---

<a id="section5"></a>

# 5. Experimentación

En los apartados anteriores de la libreta se ha realizado tanto el **preprocesamiento del conjunto de datos** - realizando una **selección de atributos** y declarando las **transformaciones** necesarias - como la **selección de modelos** - eligiendo los **modelos** a evaluar y el **conjunto de hiperparámetros** a probar para cada uno de ellos.

El siguiente paso a realizar en el proceso de la ciencia de datos es la **experimentación** - el **entrenamiento y evaluación** de los modelos seleccionados con el fin de elegir el **modelo final** a utilizar para realizar predicciones a partir de instancias nuevas de datos.

El objetivo de nuestro problema de **regresión** es encontrar el modelo que minimice la **raiz del error cuadrático medio (*Root Mean Squared Error* o *RMSE*)** - es decir, el modelo que **mejor ajusta las predicciones del tiempo de diagnóstico** a los atributos médicos de cada paciente, penalizando especialmente a los *outliers*:

$$\text{RMSE} = \sqrt{\frac{1}{N} \sum_{n=1}^{N}\left( y^{(n)} - \hat{y}^{(n)}\right)^2}$$

Donde:
- $y^{(n)}$ representa el **valor esperado** para una instancia.
- $\hat{y}^{(n)}$ representa el **valor predecido** para una instancia.

Esta sección se divide en los siguientes tres apartados seguidos para encontrar el modelo:
1. **Entrenamiento y ajuste de hiperparámetros**: Mediante un proceso de **validación cruzada** sobre el **conjunto de entrenamiento**, se entrenan todos los modelos con **sus posibles conjuntos de hiperparámetros** - seleccionando, para cada modelo, el **conjunto de hiperparámetros que minimiza el error de ajuste**.
2. **Selección del mejor modelo ajustado**: A partir de los modelos ya entrenados y ajustados, se **evalua la calidad de los modelos** sobre el **conjunto de validación** - con el fin de obtener, de forma honesta y sin fugas de datos, el **modelo generalizado con mejor resultados**.
3. **Evaluación final del mejor modelo**: Tras seleccionar el modelo definitivo, se **calcula su rendimiento real** a partir de un **conjunto de test** no estudiado previamente - evaluando su error a partir de una **plataforma externa** (`Kaggle`)

---

Esta sección de la libreta y del proceso de ciencia de datos es la más **costosa** - tanto en **tiempo** como en **recursos computacionales**. Por lo tanto, para evitar ejecuciones innecesarias o duplicadas, lo normal es **almacenar los resultados** - tanto los **modelos** como los **estadísticos** - en disco, para cargarlos posteriormente.

Aun así, la siguiente variable - `FORCE_MODEL_TRAINING` - obliga a la libreta a **realizar el entrenamiento de nuevo**, ignorando los contenidos en disco. Por defecto, su valor es `False`, pero se puede cambiar a `True` para depuración:

In [72]:
# If True, ignores already trained models on disk and runs everything
# KEEP TO FALSE UNLESS DEBUGGING OR FORCING A FRESH RUN
FORCE_MODEL_TRAINING = False

---

<a id="section5-1"></a>

## 5.1. Ajuste de hiperparámetros

El primer paso de la experimentación consiste en el **ajuste de hiperparámetros** - encontrar, para cada modelo, el **conjunto de hiperparámetros** entre todos los posibles que **maximiza el rentimiento del modelo entrenado**. En este caso - un problema de regresión -, se busca el **conjunto de hiperparámetros que minimiza el error** (es decir, **la diferencia entre el valor obtenido y el valor esperado**) a la hora de predecir un tiempo de diagnóstico para cada paciente en base a sus atributos.

Para realizar esta búsqueda, se utilizan técnicas de **validación cruzada** donde se evalua el rendimiento del modelo para **distintos conjuntos de hiperparámetros** subdividiendo el conjunto de entrenamiento en **particiones de igual tamaño** y comprobando su rendimiento para cada combinación de particiones, con el fin de **evaluar de forma honesta** el rendimiento de los hiperparámetros buscando evitar el sobreajuste - 

Concretamente, el algoritmo funciona de la siguiente manera:
1. Se divide el conjunto de datos en `n` particiones de igual tamaño - en este caso, **cinco particiones**.
2. Para cada **posible conjunto de hiperparámetros**:
    1. Para cada **partición del conjunto de datos**:
        1. Se toma la partición elegida como **conjunto de validación**, y el resto de particiones (en conjunto) como **conjunto de entrenamiento**.
        2. Se **entrena el modelo sobre el conjunto de entrenamiento**.
        3. Se **evalua el error del modelo sobre el conjunto de validación**, obteniendo una **puntuación**.
    2. Tras obtener una puntuación para cada posible combinación de conjuntos de entrenamiento y validación, se obtiene una **puntuación final para el conjunto de hiperparámetros** calculando el promedio de las puntuaciones.
3. Tras evaluar todos los posibles conjuntos de hiperparámetros, se **elige el mejor conjunto de hiperparámetros** - es decir, aquel conjunto que **ha obtenido la mejor puntuación**.

Uno de los puntos claves a resolver es la **selección de conjuntos de hiperparámetros**. Para modelos simples con pocos hiperparámetros y tiempos de entrenamiento rápidos, realizar una *búsqueda exhaustiva** de todos los posibles hiperparámetros resulta factible. Ahora bien, conforme aumenta la complejidad de los modelos - y, consecuentemente, aumenta el número de hiperparámetros y el tiempo de entrenamiento -, una búsqueda exhaustiva resulta imposible, siendo necesario algún tipo de **búsqueda aleatoria** para probar *suficientes* conjuntos de hiperparámetros.

Concretamente, se plantean las siguientes **tres opciones** para realizar la búsqueda de hiperparámetros:
- **Búsqueda exhaustiva (`GridSearch`)**: Se prueban todas las posibles combinaciones de hiperparámetros definidos.
    - Se garantiza que se encontrará la **solución óptima**. Ahora bien, el número de combinaciones a probar **crece exponencialmente** con el número de hiperparámetros, por lo que su coste se vuelve excesivo rápidamente.
- **Búsqueda aleatoria (`RandomizedSearch`)**: Se prueba **un número constante** de todas las posibles combinaciones de hiperparámetros definidos.
    - Se pueden definir **distribuciones aleatorias** para los hiperparámetros - de las que se muestrean valores - frente a la necesidad de definir valores específicos.
    - La aleatoriedad **impide garantizar una solución óptima**. En cambio, el número **constante** de subconjuntos a probar hace que **no crezca el coste de la búsqueda** con el número de hiperparámetros - creciendo únicamente si se aumenta el número de subconjuntos a probar.
    - Dado un **número suficientemente grande de subconjuntos probados**, el algoritmo encuentra con alta probabilidad una solución suficientemente buena.
- **Búsqueda aleatoria guiada (`BayesSearch`)**: Una evolución del paradigma anterior en la que se sigue probando **un número constante** de todas las posibles combinaciones de hiperparámetros definidos. Ahora bien, se utiliza un algoritmo de **optimización bayesiana** subyacente para guiar la búsqueda aleatoria de forma probabilística.
    - El modelo de optimización **añade un sobrecoste a la búsqueda** - al entrenarse un segundo modelo, se añade un coste adicional de tiempo y recursos.
    - El algoritmo cuenta con los mismos problemas y ventajas que la búsqueda aleatoria. La principal diferencia radica en que **es necesario un número menor de búsquedas para encontrar un conjunto de hiperparámetros suficientemente bueno** - siendo la principal ventaja de este método, por tanto, el entrenamiento de **modelos muy complejos con tiempos de entrenamiento largos**, donde reducir el número de entrenamientos es el objetivo principal.
    
Durante la sección anterior, se definió para cada modelo una **matriz de posibles hiperparámetros** y el **tipo de búsqueda a realizar** - siendo las siguientes:

In [35]:
# Extract the models per type of required training
grid_cv_models = [model_name for model_name in model_pipelines if model_pipelines[model_name]["hyperparameter_search"]=="grid"]
randomized_cv_models = [model_name for model_name in model_pipelines if model_pipelines[model_name]["hyperparameter_search"]=="random"]
bayes_cv_models = [model_name for model_name in model_pipelines if model_pipelines[model_name]["hyperparameter_search"]=="bayes"]

# Print the list of models
print(f"Models with Grid CV: {grid_cv_models}")
print(f"Models with Random CV: {randomized_cv_models}")
print(f"Models with Bayes CV: {bayes_cv_models}")

Models with Grid CV: ['linear_regression', 'ridge_l2', 'lasso_l1', 'elastic-net']
Models with Random CV: ['decision_tree', 'svm_linear', 'svm_poly', 'svm_rbf', 'svm_sigmoid']
Models with Bayes CV: ['random_forest', 'extra_trees', 'adaboost', 'xgboost', 'catboost', 'lgbm', 'hstgradientboost']


---

### 5.1.1. Preparación para la búsqueda

El proceso de experimentación es complejo - con un gran número de **modelos**, **hiperparámetros** y **subconjuntos de atributos** a probar.

Con el fin de **automatizar** este proceso al máximo - y, de esta manera, reducir los posibles **puntos de fallo** reutilizando la mayor cantidad de código posible -, se opta por definir previamente una serie de **elementos** antes de iniciar la búsqueda:

#### **Estructuras de datos**

Para almacenar todos los artefactos generados durante el ajuste de hiperparámetros - **modelos entrenados** y **estadísticos resultantes del entrenamiento** -, se crean o cargan varias estructuras de datos para almacenarlos:

##### - `dict_training_results`: 

Un diccionario que almacena, para cada **modelo** y **subconjunto de atributos**, los resultados principales del proceso de **ajuste de hiperparámetros**. Este diccionario sigue la siguiente estructura:

```
dict_training_results: {
    <NOMBRE DEL MODELO>: {
        <SUBCONJUNTO DE ATRIBUTOS UTILIZADO>: {
            "hyperparameters" -> {DICCIONARIO DE HIPERPARÁMETROS},
            "cv_total_time" -> <TIEMPO DE ENTRENAMIENTO TOTAL(segs)>,
            "cv_iters" -> <NÚMERO DE ITERACIONES DE VALIDACION CRUZADA>,
            "cv_time_per_iter" -> <TIEMPO APROXIMADO POR ITERACION>,
            "training_time" -> <TIEMPO DE ENTRENAMIENTO FINAL(segs)>
            "training_score" -> <RMSE DEL MODELO ELEGIDO>
        },
        ...
    },
    ...
}
```

Donde:
- `hyperparameters`: Un diccionario con los **mejores hiperparámetros** para el modelo y el subconjunto de atributos - es decir, los **parámetros que minimizan el error**.
- `cv_total_time`: El **tiempo de entrenamiento total**, en segundos. Este tiempo de entrenamiento incluye:
    - El entrenamiento de **cinco modelos** - uno para cada partición.
    - El entrenamiento del **modelo final** sobre el conjunto de datos completos.
    - En caso de que fuese necesario, el **sobrecoste** de la búsqueda de hiperparámetros.
- `cv_iters`: El **número de iteraciones** de validación cruzada realizadas.
- `cv_time_per_iter`: El **tiempo de entrenamiento por iteración**, aproximado.
- `training_time`: El **tiempo de entrenamiento del modelo final**, en segundos.
- `training_score`: La **puntuación** del modelo con los mejores hiperparámetros - en este caso, el ***RMSE*** del modelo.
    - Esta puntuación no está calculada sobre el conjunto de validación separado - sino como **un promedio de la puntuación de los cinco modelos entrenados durante la validación cruzada**.

El objetivo de este diccionario es únicamente **almacenar la información estadística del entrenamiento** - para poder estudiar posteriormente el comportamiento de los modelos durante el ajuste. Para esto, se transformará posteriormente el diccionario en un *DataFrame* - `df_training_results` -, con el fin de facilitar dicho estudio.

Esta información se almacenará además en disco, en el fichero `results/training_results.csv`. En el caso de que el fichero existiese de antemano, **se carga** en lugar de realizar la búsqueda de hiperparámetros posterior:


In [62]:
dict_training_results = {}
df_training_results = None

# Create the folder - if it does not exist
Path(r"./results").mkdir(parents=True, exist_ok=True)

# If appropriate - load the information and transform it into a dictionary
if not FORCE_MODEL_TRAINING:
    pass

##### - `trained_models`: 

Un diccionario almacenando, para cada posible algoritmo, el **mejor modelo obtenido del proceso de validación cruzada** - tras ajustar sus **hiperparámetros** y **elegir un subconjunto de atributos** - y su información. El diccionario tiene la siguiente estructura:

```
trained_models: {
    <NOMBRE DEL MODELO>: {
        "model": <MODELO ENTRENADO>,
        "information": {
            "feature_subset": <SUBCONJUNTO DE ATRIBUTOS>,
            "hyperparameters" -> {DICCIONARIO DE HIPERPARÁMETROS},
            "cv_total_time" -> <TIEMPO DE ENTRENAMIENTO TOTAL(segs)>,
            "training_time" -> <TIEMPO DE ENTRENAMIENTO FINAL(segs)>
            "training_score" -> <RMSE DEL MODELO ELEGIDO>
        }
    },
    ...
}
```

Donde:
- `feature_subset`: El **subconjunto de atributos** elegido para el modelo tras el ajuste de hiperparámetros.
- `hyperparameters`: Un diccionario con los **mejores hiperparámetros** para el modelo y el subconjunto de atributos - es decir, los **parámetros que minimizan el error**.
- `cv_total_time`: El **tiempo de entrenamiento total**, en segundos. Este tiempo de entrenamiento incluye:
    - El entrenamiento de **cinco modelos** - uno para cada partición.
    - El entrenamiento del **modelo final** sobre el conjunto de datos completos.
    - En caso de que fuese necesario, el **sobrecoste** de la búsqueda de hiperparámetros.
- `training_time`: El **tiempo de entrenamiento del modelo final**, en segundos.
- `training_score`: La **puntuación** del modelo con los mejores hiperparámetros - en este caso, el ***RMSE*** del modelo.
    - Esta puntuación no está calculada sobre el conjunto de validación separado - sino como **un promedio de la puntuación de los cinco modelos entrenados durante la validación cruzada**.

La estructura del diccionario es similar a la de `dict_training_results` - la principal diferencia se encuentra en que **se almacenan los modelos** y la **información específica para el modelo elegido** - a utilizar durante los pasos posteriores de **validación**.

Esta información es almacenada en disco por partida doble:
- Los modelos entrenados son almacenados en ficheros con la estructura `models/<model_name>/model.pkl`. 
- La información del entrenamiento se almacena en un fichero *JSON* con la estructura `models/<model_name>/info.json`.


Si los modelos ya han sido entrenados previamente (se encuentran **almacenados en memoria**), el diccionario se inicializa **cargando los modelos entrenados desde los archivos donde se almacenan** - evitando posteriormente realizar un entrenamiento duplicado. En cualquier otro caso, el diccionario comienza vacío y se entrenan los modelos de forma esperada.

In [123]:
trained_models = {}

# Check if the folder exists and create it
Path(r"./models/").mkdir(parents=True, exist_ok=True)

# If applicable, load all available models
if not FORCE_MODEL_TRAINING:
    for model_name in os.listdir(r"./models/"):
        pass

#### **Constantes**

##### - Agrupamiento de atributos categóricos

Como se describió durante el preprocesamiento, la mayoría de los modelos a entrenar necesitan **codificar sus atributos categóricos** utilizando *One-Hot Encoding*. Ahora bien, los atributos categóricos del conjunto de datos presentan una **alta complejidad** - con una gran cantidad de posibles valores y con presencias desequilibradas en el conjunto de datos.

Para tratar esta complejidad, es necesario realizar un **agrupamiento de los valores** - juntar los **valores menos frecuentes** bajo un único valor comun (`Infrecuente`). Ahora bien, **el umbral** en el que se considera que un valor es infrecuente es también **un posible hiperparámetro del modelo a ajustar** - y, por tanto, necesita ser parte del **proceso de búsqueda**.

En vez de añadir directamente los valores en cada uno de los diccionarios declarados en la sección anterior, se opta por **crear una única lista de posibles valores** - para facilitar la experimentación y la modificación de los valores. Esta lista se añade directamente antes de realizar la experimentación al conjunto de hiperparámetros:

In [122]:
# Possible hyperparameters for the umbral at which values of categorical variables are GROUPED
# (if a value has less than the specified percentage of instances, it is grouped as a generic Other category instead)
HYPERPARAMETERS_ONEHOT_GRID = [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3]
HYPERPARAMETERS_ONEHOT_RANDOMIZED = Real(0.0, 0.3)
HYPERPARAMETERS_ONEHOT_BAYES = Real(0.0, 0.3)

#### **Métodos**

En general, el proceso de búsqueda de hiperparámetros es muy parecido entre todos los tipos de búsqueda a realizar - **exhaustiva**, **aleatoria** y **guiada**. Ahora bien, hay algunas diferencias clave en las **clases e implementaciones** a utilizar para cada búsqueda que es necesario considerar.

Para poder reutilizar el código entre todas las búsquedas y facilitar su mantenimiento, se ha creado:
- Un **método principal** encargado de realizar el ajuste de hiperparámetros y la escritura de los datos en las estructuras de datos adecuadas.
- **Métodos auxiliares** encargados de realizar las tareas anciliares - carga y guardado de modelos en disco, selección de hiperparámetros...

##### - Métodos auxiliares

Los siguientes tres métodos se encargan de realizar tareas auxiliares a la búsqueda principal de hiperparámetros:
- `get_best_feature_subset()`: Dado un diccionario con los resultados para cada subconjunto de atributos, **devuelve el mejor subconjunto**.
- `load_model()` y `save_model()`: Lee y almacena, respectivamente, el **modelo** y sus **hiperparámetros / estadísticas** en disco.

In [119]:
def get_best_feature_subset(feature_subset_statistics):
    """Given all the statistics for each feature subset, extract the subset with the best score (lowest error)"""

    # Extract, for each feature subset, its associated score
    scores = {fs: feature_subset_statistics[fs]["training_score"] for fs in feature_subset_statistics}
    
    # Return the feature subset with the lowest error
    return min(scores, key=scores.get)

In [120]:
def load_model(model_name):
    """Loads a model and its hyperparameters / statistics from disk"""

    # Load the model
    with open(rf"./models/{model_name}/model.pkl", "rb") as model_file:
        model = pickle.load(model_file)
    # Load the statistics
    with open(rf"./models/{model_name}/info.json", "r") as json_file:
        info = json.load(json_file)    

    return model, info

In [121]:
def save_model(model_name, model, info):
    """Given a model and its hyperparameters / statistics, save it into disk"""
    # Create the path
    Path(rf"./models/{model_name}/").mkdir(parents=True, exist_ok=True)

    # Store the model
    with open(rf"./models/{model_name}/model.pkl", "wb") as model_file:
        pickle.dump(
            obj=model,
            file=model_file,
            protocol=5
        )
    # store the statistics
    with open(rf"./models/{model_name}/info.json", "w") as json_file:
        json.dump(
            obj=info,
            fp=json_file
        )

##### - `hyperparameter_search`

`hyperparameter_search` es el **método principal de ajuste de hiperparámetros**, con las siguientes características:
- **Entrada**:
    - `model_list`: Una lista con los **modelos a considerar** - extraidos de `model_pipelines`.
    - `model_pipelines`: Un **diccionario** conteniendo, para cada **modelo** y **subconjunto de datos**, el *Pipeline* preparado con preprocesamiento y modelo.
        - Esta estructura se generó durante la sección anterior.
    - `search_type`: El **tipo de búsqueda** que se va a realizar:
        - `"grid"`: Búsqueda exhaustiva.
        - `"randomized"`: Búsqueda aleatoria.
        - `"bayes"`: Búsqueda aleatoria guiada.
    - `n_iters`: Para las búsquedas aleatorias, **número total de conjuntos de hiperparámetros a comprobar**. Ignorado por las búsquedas exhaustivas.
    - `trained_models`: Un **diccionario** conteniendo, para cada **modelo**, el **mejor modelo entrenado** junto a sus hiperparámetros y estadísticos.
        - Esta estructura se ha generado en esta sección, y será completada durante el método.
    - `dict_training_results`: Un **diccionario** conteniendo, para cada **modelo** y **subconjunto de atributos** los **hiperparámetros** que mejor resultado han tenido y estadísticos del entrenamiento.
        - Esta estructura se ha generado en esta sección, y será completada durante el método.
- **Salida**:
    - `trained_models`, actualizado con los modelos seleccionados y entrenados.
    - `dict_training_results`, actualizado con los modelos seleccionados y entrenados.

In [124]:
def hyperparameter_search(model_list, model_pipelines, search_type, n_iters, trained_models, dict_training_results):
    """
    Performs a hyperparameter search of the specified type, with the specified number of iterations

    Parameters
    ---------
    model_list: list[str]
        List containing the name of the models to evaluate
    model_pipelines: dict
        Dictionary containing the UNTRAINED pipelines for each model and feature subset
    search_type: "grid" | "randomized" | "bayes"
        Type of hyperparameter search to perform
    n_iters: int
        Number of iterations to perform. Only applicable to randomized and bayes searches
    trained_models: dict
        Dictionary containing the selected, TRAINED model and its hyperparameters / statistics
    dict_training_results: dict
        Dictionary containing the hyperparameters / statistics for every model and feature subset
    
    Returns
    -------
    (dict, dict)
        Trained models and dict_training_results, updated
    """
    
    # Ensure that a search type is properly specified
    if search_type not in ["grid", "randomized", "bayes"]:
        raise Exception("Unknown search type")

    # Time keeping - for printing
    current_time = time()

    # Perform a search for each specified model
    for model_name in model_list:

        print(f"== MODEL {model_name} ({(time() - current_time):.2f}s)")
        
        # If the model already exists, and if appropriate - load it instead of training it
        if os.path.exists(rf"./models/{model_name}/model.pkl") and os.path.exists(rf"./models/{model_name}/info.json") and not FORCE_MODEL_TRAINING:
            print(f"- Loading model {model_name} from disk ({(time() - current_time):.2f}s)")

            model, info = load_model(model_name)
            trained_models[model_name] = {
                "model": model,
                "information": info
            }

            print(f"- Loaded model {model_name} from disk ({(time() - current_time):.2f}s)")
            continue

        else:
            # Extract the hyperparameter list and, if the categorical variables use one-hot encoding,
            # add the appropriate hyperparameters
            hyperparameters = model_pipelines[model_name]["hyperparameter_grid"]
            if model_pipelines[model_name]["categorical_encoding"] == "one-hot":
                if search_type == "grid":
                    hyperparameters["preprocessing__cat__oh__min_frequency"] = HYPERPARAMETERS_ONEHOT_GRID
                elif search_type == "randomized":
                    hyperparameters["preprocessing__cat__oh__min_frequency"] = HYPERPARAMETERS_ONEHOT_RANDOMIZED
                elif search_type == "bayes":
                    hyperparameters["preprocessing__cat__oh__min_frequency"] = HYPERPARAMETERS_ONEHOT_BAYES

            # Prepare the results dictionary to contain the results for each subset of features
            dict_training_results[model_name] = {}

            # Create a dictionary to temporarily store the trained models - before selection
            models = {}

            # Each subset of features has to be compared separately
            for feature_subset, pipeline in model_pipelines[model_name]["models"].items():

                # Time keeping
                training_time = time()
                print(f"-Training model {model_name} on feature subset {feature_subset} ({(time() - current_time):.2f}s)")

                # Create the cross-validation model using RMSE
                # Since Scikit maximizes by default, the score is negated

                # The chosen cross-validation model depends on the specified search type
                if search_type == "grid":
                    cv = GridSearchCV(
                        estimator=pipeline,
                        param_grid=hyperparameters,
                        scoring="neg_root_mean_squared_error"
                    )
                elif search_type == "randomized":
                    cv = RandomizedSearchCV(
                        estimator=pipeline,
                        param_distributions=hyperparameters,
                        n_iter=n_iters,
                        scoring="neg_root_mean_squared_error",
                        random_state=RANDOM_SEED
                    )
                elif search_type == "bayes":
                    cv = BayesSearchCV(
                        estimator=pipeline,
                        search_spaces=hyperparameters,
                        n_iter=n_iters,
                        scoring="neg_root_mean_squared_error",
                        random_state=RANDOM_SEED
                    )

                # Fit the model
                cv.fit(X_train, y_train)

                # Update the training time
                current_time = time() - training_time
                print(f"-Trained model {model_name} on feature subset {feature_subset} ({(time() - current_time):.2f}s)")
                
                # Compute additional statistics
                final_n_iters = len(list(itertools.product(*hyperparameters.values()))) if search_type == "bayes" else n_iters

                # Store the information into the dictionary and store the model temporarily
                # NOTE - The score is given as a negative number, and must be turned into a positive one
                dict_training_results[model_name][feature_subset] = {
                    "hyperparameters": cv.best_params_,
                    "cv_total_time": current_time,
                    "cv_iters": final_n_iters,
                    "cv_time_per_iter": current_time / final_n_iters,
                    "training_time": cv.refit_time_,
                    "training_score": abs(cv.best_score_)
                }
                models[feature_subset] = cv.best_estimator_
            
            # After training the model on all feature subsets, select the one with the best (lowest) error
            selected_fs = get_best_feature_subset(dict_training_results[model_name])

            # Extract the details about the selected feature subset model
            results_dict = {
                "feature_subset": selected_fs,
                "hyperparameters": dict_training_results[model_name][selected_fs]["hyperparameters"],
                "cv_total_time": dict_training_results[model_name][selected_fs]["cv_total_time"],
                "training_time": dict_training_results[model_name][selected_fs]["training_time"],
                "training_score": dict_training_results[model_name][selected_fs]["training_score"],
            }

            # Store the trained model and information into the dictionary, for the next step of data science
            trained_models[model_name] = {
                "model": models[selected_fs],
                "information": results_dict
            }

            # Store the model and information into disk
            save_model(
                model_name=model_name, 
                model=models[selected_fs], 
                info=results_dict
            )

            print(f"-Stored model {model_name} based on feature subset {selected_fs} ({(time() - current_time):.2f}s)")
    
    return trained_models, dict_training_results

---

---

---

### 5.1.2. Búsqueda de hiperparámetros exhaustiva - `GridSearchCV`

TODO ESCRIBE ALGO AQUI
TODO CARGA AUTOMATICAMENTE MODELOS Y DATOS AL DICCIONARIO SI YA SE HA ENTRENADO PREVIAMENTE

== MODEL linear_regression (0.00s)
-Training model linear_regression on feature subset manual (0.00s)
-Trained model linear_regression on feature subset manual (2.49s)
-Training model linear_regression on feature subset filter (2.49s)
-Trained model linear_regression on feature subset filter (5.53s)
-Training model linear_regression on feature subset wrapper (5.53s)
-Trained model linear_regression on feature subset wrapper (8.11s)
-Training model linear_regression on feature subset no_selection (8.11s)
-Trained model linear_regression on feature subset no_selection (26.32s)
-Stored model linear_regression based on feature subset wrapper (26.34s)
== MODEL ridge_l2 (26.34s)
-Training model ridge_l2 on feature subset manual (26.34s)
-Trained model ridge_l2 on feature subset manual (60.11s)
-Training model ridge_l2 on feature subset filter (60.11s)
-Trained model ridge_l2 on feature subset filter (99.13s)
-Training model ridge_l2 on feature subset wrapper (99.13s)
-Trained model ridge_l2 

### 5.1.3. Búsqueda de hiperparámetros aleatoria - `RandomizedSearchCV`

In [ ]:
# TODO - UPDATE DATA DISTRIBUTIONS

# RANDOMIZED HYPERPARAMETER SEARCH - RANDOMIZED SEARCH
# Perform the search for each model using grid search
for model_name in randomized_cv_models:
    
    # Extract the hyperparameter list and, if the categorical variables use one-hot encoding,
    # add the appropriate hyperparameters
    hyperparameters = model_pipelines[model_name]["hyperparameter_grid"]
    if model_pipelines[model_name]["categorical_encoding"] == "one-hot":
        hyperparameters["preprocessing__cat__oh__min_frequency"] = HYPERPARAMETERS_ONEHOT_RANDOMIZED

    # Prepare the results dictionary to contain the results for each subset of features
    dict_hyperparameter_search_results[model_name] = {}

    # Each subset of features has to be compared separately
    for feature_subset, pipeline in model_pipelines[model_name]["models"].items():
        
        # Name for model storage
        model_name = f"{model_name}_{feature_subset}"
        # Time keeping
        current_time = time()

        # Create the cross-validation model using RMSE
        cv = RandomizedSearchCV(
            n_iter=50,
            estimator=pipeline,
            param_distributions=hyperparameters,
            scoring="neg_root_mean_squared_error",
            random_state=RANDOM_SEED
        )       

        # Fit the model
        cv.fit(X_train, y_train)

        # Update the training time
        current_time = time() - current_time
        
        # Update the dictionary with the statistics of the training
        # NOTE - The score is given as a negative number, and must be turned into a positive one
        dict_hyperparameter_search_results[model_name][feature_subset] = {
            "model": cv.best_estimator_,
            "hyperparameters": cv.best_params_,
            "cv_time": current_time,
            "training_time": cv.refit_time_,
            "training_score": abs(cv.best_score_)
        }

ValueError: 
All the 250 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "d:\Users\Luna\miniforge3\envs\cidaen\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "d:\Users\Luna\miniforge3\envs\cidaen\Lib\site-packages\sklearn\base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Users\Luna\miniforge3\envs\cidaen\Lib\site-packages\sklearn\pipeline.py", line 654, in fit
    Xt = self._fit(X, y, routed_params, raw_params=params)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Users\Luna\miniforge3\envs\cida

### 5.1.4. Búsqueda de hiperparámetros aleatoria guiada - `BayesSearchCV`

---

### 5.1.5. Almacenamiento de los resultados

In [101]:
# Transform the dictionary into a DataFrame
dict_training_results_reshaped = {
    (model_name, feature_subset): data
    for model_name, model_dict in dict_training_results.items()
    for feature_subset, data in model_dict.items()
}
df_training_results = pd.DataFrame(dict_training_results_reshaped).transpose()
display(df_training_results)


hyperparameters  \
linear_regression manual          {'preprocessing__cat__oh__min_frequency': 0.01}   
                  filter          {'preprocessing__cat__oh__min_frequency': 0.01}   
                  wrapper         {'preprocessing__cat__oh__min_frequency': 0.01}   
                  no_selection    {'preprocessing__cat__oh__min_frequency': 0.01}   
ridge_l2          manual        {'preprocessing__cat__oh__min_frequency': 0.01...   
                  filter        {'preprocessing__cat__oh__min_frequency': 0.01...   
                  wrapper       {'preprocessing__cat__oh__min_frequency': 0.01...   
                  no_selection  {'preprocessing__cat__oh__min_frequency': 0.01...   

                                  cv_time training_time training_score  
linear_regression manual         2.493994      0.061499      85.592024  
                  filter           3.0355      0.077499      85.456956  
                  wrapper        2.577502         0.075       85.35181  
                  no_selection  18.214499        0.7775       86.28996  
ridge_l2          manual        33.773952         0.065      85.584336  
                  filter           39.022      0.087499      85.449198  
                  wrapper       34.570501      0.076996        85.3506  
                  no_selection    159.847         0.416      85.958689

---

<a id="section5-2"></a>

## 5.2. Validación de los modelos ajustados 

TODO EXPLICA AQUI EL CONCEPTO DE FUGA DE DATOS?

---

<a id="section5-3"></a>

## 5.3. Evaluación del modelo final contra el conjunto de test




- XGBOOST PUEDE TENER EARLY STOPPING CON VALIDACION - HAY QUE INDICAR GPU Y TENER CUIDADO CON EL THREADING

---

<a id="section6"></a>

# 6. Análisis de resultados